<a href="https://colab.research.google.com/github/andyarnell/WDPA_designation_overlap/blob/master/GEE_python_AOH_multiband_v001.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# AIM: to convert a feature collection of IUCN Red List species' range data into an image collection - including ability to refine to area of Habitat (AOH).

**Written by**: Andy Arnell 
Based on Code Editor scripts by: Andy Arnell and Corinna Ravilious
Additions: Martin Jung and Alison Eyers

Adapted version
https://code.earthengine.google.com/?scriptPath=users%2Fcorinnaravilious%2FUNEP-WCMC_SharedScripts%3Ap8425_NatureMap%2FAOH%2Fpython_AOH_test1

**Last update**: 20.05.2021
**Version**: 001

For help converting code from JavaScript to Python see: https://developers.google.com/earth-engine/python_install

**Tip**: Before editing the python field, try executing small code in a Scratch cell (CTRL+ALT+N)

# Import the API, authenticate and initialize
Import the API into your session, run the `ee.Authenticate` function to authenticate your access to Earth Engine servers, and then run `ee.Initialize` to initialize it. Upon running the following cell you'll be asked to grant Earth Engine access to your Google account. Follow the instructions printed to the cell.
These steps must be completed for each new Colab session, if you restart your Colab kernel, or if your Colab virtual machine is recycled due to inactivity.

In [1]:
try:
  # Import Google Earth Engine python module
  import ee
  # Trigger the authentication flow.
  ee.Authenticate()
  # Initialize the library.
  ee.Initialize()
  print(ee.__version__)
  
except ImportError:
  print('Install the python earthengine-api module first!')

To authorize access needed by Earth Engine, open the following URL in a web browser and follow the instructions. If the web browser does not start automatically, please manually browse the URL below.

    https://accounts.google.com/o/oauth2/auth?client_id=517222506229-vsmmajv00ul0bs7p89v5m89qs8eb9359.apps.googleusercontent.com&scope=https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fearthengine+https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdevstorage.full_control&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&response_type=code&code_challenge=AzmHe9W1FhNl08KJnmpfG64uLUB5h-tNFVwxvGCnKcM&code_challenge_method=S256

The authorization workflow will generate a code, which you should paste in the box below. 
Enter verification code: 4/1AY0e-g7UPeyzbW1s_OWI7_VJtAsGCUliUdSucvT4JxEBGFKd1pV6dhLsSQ0

Successfully saved authorization token.
0.1.264


#Importing packages

In [458]:
import ee                               # The Earth Engine package.
import folium                            # For displaying interactive maps 
import math, os, datetime, csv, time    # Basic tools
import IPython.display                  # For plotting
import pandas as pd                     # Data analysis package.
from pprint import pprint               # For pretty-printing data structures.
from tqdm.notebook import tnrange, tqdm # For the progress bar
# import altair as alt                    # Graphing package.
# import json                             # For handling JSON files
# import matplotlib.pyplot as plt         # Import the matplotlib.pyplot module.
#for loading data table
%load_ext google.colab.data_table
print("version",str(ee.__version__))

The google.colab.data_table extension is already loaded. To reload it, use:
  %reload_ext google.colab.data_table
version 0.1.264


#Setting parameters


In [457]:

# Taxonomic groups to process and input data to process
# Data sources are loaded in further down below
IUCN_version = '2019_2' # This defines which version of the assets are loded in further down below

target_group = ['AMPHIBIA','MAMMALIA','AVES','MultiTax'][3] # Use index to select different one, possible range [0-5]

###### Export parameters 

toDrive = False# Should the export be done to Google Drive True or False

#if to asset
targetImageCollName = "projects/unep-wcmc/p08276_GIMM/work_in_progress/aoh_30arcsec" #TO DO add code to create image collection if doesnt exist
skipIfAssetExists = True# as it sounds like. Saves lots of red in Tasks list in code editor

#if to drive
folderName = "GIMM_30arcsec_tifs" # if exporting to google drive 

#as part of the output image name
prefix = "" # prefix to be inserted in name of image

suffix = "pnvV004" # suffix to be inserted in name of image

######### Run parameters

debug = True # Setting this to true will output a lot of print messages in the main processing

startNumber = 0 #should be 0 unless offsetting the start e.g., if some images have exported, but if was stopped/timed out you can set this to start from the next batch
exportMaxNumber =  100000 # Maximum number of AOH to be exported. if running all comprehensively assessed RL species, this should be a large number (100,000)
numberOfBandsPerImage = 50 # Batch size to be computed (batches of 50 run quite fast, when running with imageCollection, but havent tested extensively)

### Export extent

staticExportRegion = False # if True then export extent is a set size (defaults to global extent*), else uses input feature bounds. 
defaultExportRegion = ee.Geometry.Rectangle([-180, -90, 180, 90], None, False); # for global extent - this can be changed to say a study area using ".bounds()" etc. but havent coded it
if debug: print ("Export bounds: ",str(defaultExportRegion.getInfo()))

##### Habitat refinement parameters 

refineByHabitatOption = ["habRefine","noHabRefine"][0] #choice to refine by habitats - if noHabRefine will attmept backup habitat map

refineByElevationOption = ["elevRefine","noElevRefine"][0] #choice to refine by elevation - requires the LUT to have elevation included obviously

habMapFormat = ["image","imageColl"][0] #habitat map format: image (categorical) or image collection (proportion of habitat class per pixel)

habMapInputChoice = ["potentialNaturalVegetation","habMap","altHabMap"][0] #do you want to use the habitat map or PNV equivalent (i.e. before human influence) or alternative map

#### Specific options for IUCN habitat map 
habMapLevel = ["level1","level2","lvl1WithLvl2Exceptions"][0] # Should the map be refined at IUCN habitats level 1 or level 2

#artificial classes - if running at level 1, some level 2 may still be worth splitting out. See notes from Paz Duran on this. 
##NB currently recoding input habitats (except those in the exceptions list, to level 1 then back to level 2, adding in the level 2 exceptions and refining based on the level 2 map)
lvl2toLvl1ExceptionsList = ee.List([1400,1401,1402,1403,1404,1405]) #1406 is not mapped in Jung habmap, for info see https://www.iucnredlist.org/resources/habitat-classification-scheme

#if habMapFormat is an image collection
imagePrefixInHabMapImageColl = "htclass" #TO DO - check if new hab map image coll has htclass as a property - if so can skip using "system:index" and contatnated strings to select images

habMapProperty = "system:index"

# Habitat type map version -> http://doi.org/10.5281/zenodo.3925749 
ht_version = '004' # Which version of the habitat type layer to use - Martin Jung set this up, possibly fro when there were multipl
## N.B. alternative maps can be used (option below), but the habitat preference look up table(LUT) needs to link to the map's classes (i.e. if a crosswalk is required from IUCN habitats to land cover types, at present, this should be ran before hand)

#alternative image/ image collection for refining
altHabMap_1kmfrac = "projects/unep-wcmc/imported/landcover/esa_cci/ESACCI-LC-L4-LCCS-Map-300m-P1Y-v207"

altHabMap_image = "projects/unep-wcmc/imported/landcover/esa_cci/ESACCI-LC-L4-LCCS-Map-300m-P1Y-v207"##NB if running scenarios this can be a collection

#back up AOH image
##TO DO - add a try except clause to refineByHabitats function to catch other issues
backupImageCreationOption = ["fromList", "fromImage"][1] #do you want to make a backup image based upon uniform values or based up on refinining by a set list of habitats

#if back up map "fromImage" 
#values to make a backup image when using image collection refinement or reprojectingOutput. 
#This is based upon a constant image value - likely 1000 or 10000 (but could use max val of 65,535 for unsigned 16 bit integer)
maxHabMapCollectionVal = 10000 #max value in habitat map image collection (in a given pixel when all summed) - used as backup image when no LUT values given

####Reprojecting

reprojectOutput = True #reproject output to a lower resolution True/False

templateImageChoice = ["templateImage1km","templateImage10km"][0] #if reprojectOutput is true what to align to - if false this is ignored

maxValueInReprojectedOutput = maxHabMapCollectionVal # leave as maxHabMapCollectionVal if want to align with image collection max values

reducerChoice = ee.Reducer.mean() ##for aggregating to coarser resolution. [UPDATE - issues on range edges but only if exporting images to drive. when doing this use mean and multiply by it's mask. Or else change to sum reducer]

######Scenarios - different time periods etc. 

runScenarios = False #do you want to run multiple scenarios (True/False)

#list the scenarios - NB need to have input as an image collection (or create one on the fly) with corresponding properties to filter
if runScenarios:
  habMapScenarioList = ["2000","2005","2010","2015"]# optional list of strings to run different scenarios. Sting used to filter image collection 
else: 
  habMapScenarioList = ["NA"]#when not runnign scenarios - leave as a single [NA] - just runs through once and without filtering inputs.

#####Filtering species

applyIUCNfiltering = True ##True will filter by Presence Origin Season Category and Class. No filtering of habitat LUTs but not needed (unless it's a a huge input table and slowing processing down)

filterTasksBySpeciesList = True# extra filtering of species - if True will filter to run subset of species using a list (see "Extra filtering" section) * 
filterTasksField = ["speciesId","idSeasonCombined","NA"][0] #what field in the spatial data to apply the filter list: i.e., by id_no or id_no-season combination. Select from list using index - if not filtering this is ignored and can be set to "NA"

####Look-up Table (LUT) for habitat affilation - assumes you have LUT with habitats and elevation

#habitats
#select look up table format: 
lutFormat = ["long","wide"][1] #long - habitats in single field, with multiple rows per species-season combination, wide - a delimited list of habitats in a single field, with a single row per species-season combination
delimiter = ";" #when format is "wide" - character used in concatonated hab/lc field #TO BE DEPRECATED 
lutHabField = ["habMapLevel2","conc"][1] #NB "habMapLevel2" for long format in most instances, "conc" field name from old wide format (should be deprecated)

lutSpeciesId = "id_no"#column name for species id tpyically "id_no" but could also be "iucn_id" 
lutSeasonCode = "seasonal"#"season" ##TO DO-  Could add an option to only use id_no for matching to species and ignore season

lutChoice = ["lut_iucn","lut_alternative"][0] #which LUT to use

minValField = "min_alt" # field name in habitats LUT from for refining by min altitude - missing values should be recoded to large e.g. 999999 values [could be a separate LUT for elevation ranges, but currently from habitats LUT]
maxValField = "max_alt" # field name in habitats LUT from for refining by max altitude - missing values should be recoded to large e.g. 999999 values

lutAlternativeAsset = "projects/unep-wcmc/p08049_trade_hub/raw/persistence_score/habitat_lut/sp_esa_cci_lcs_suit_small"

#main fields in species spatial data (featureCollection)
speciesId = "id_no" # CHANGE TO speciesID FIELD  main column to put in image
seasonCode = "seasonal" # CHANGE TO seasonalCode field - seasonal aspect 
idSeasonCombined = "id_seas" # "id_seas" - name to call the (speciesId and seasonCode concatonated) column

###### Elevation data parameters

elevFormat = ["image","imageColl"][0] #elevation format - DEM image or image collection with proportions of elevation slices (e.g. 100-200m, 200-300m etc)

maxElevCollectionVal = 10000 #if elevFormat is imagecoll, max value in collection - used as backup image when no LUT values given

demMapChoice = ["demImage250m","demImage1km","demImageCollection1km","demImageCollection10km"][0] #choose res option if using dem image

#### Filtering Red List spatial data - parameters

# Field selection for filtering
presenceField = "presence" 
originField = "origin" 
seasonField = "seasonal"
categoryField = "category"
taxPrefix = "class" # useful for filtering - BUT MAYBE REMOVE?

##choices
#automatic switch depending on input type (could be more elegantly coded)
if habMapInputChoice == "potentialNaturalVegetation":  
  presenceValsListIndex = 1 
else: 
  presenceValsListIndex = 0

presenceValsList = [[1,4],[1,4,5,6]][presenceValsListIndex] #list of lists - automatically sets 1 for PNV map historic and 0 if 
originValsList  = [1,2]
seasonalValsList = [1,2,3,4,5]
categoryValsList = ["CR","EN","VU","DD","NT","LC","EW","EX"][0:4] #INDEX selection of status categories - list of possible categories - SEE ALSO LINE BELOW <NB NEED TO ADD THE OTHERS FROM REPTILES LC/nt / LC/ etc.>


#<TO ADD IF TIME - export paramter choices to the images themselves as a new property string of metadata, and/or to an external csv/ asset >
#----------------------------------------------------------------------------

##defining allHabsList currently needs a list of habitats to attempt to limit errors - try, except statement could bypass this, but useful for recoding between levels still
if habMapLevel == "level1":
  allHabsList = ee.List(
  [100,200,300,400,500,600,
  ##700,
  800,
  900,1000,1100,1200, # Marine stuff
  1400 ##1500,1600,1700
  ]);
elif habMapLevel == "level2" or "lvl1WithLvl2Exceptions":
  allHabsList = ee.List(
  [100,101,102,103,104,105,106,107,108,109,
   200,201,202,
   300,301,302,303,304,305,306,307,308,
   400,401,402,403,404,405,406,407,
   500,501,502,503,504,505,506,507,508,509,510,511,512,513,514,515,516,517,518,
   600,
   ##700,701,702,
   800,801,802,803,
   900,901,908,909, ##902,903,904,905,906,907,980,981,982,983,984,985,986,909,910,// Marine stuff
   1000,1001,1002,1003,1004, ##Marine stuff
   1100,1101,1102,1103,1104,1105,1106, ## Marine stuff
   1200,1206,1207, ##Marine stuff
   #1300,1301,1302,1303,1304,1305,
   1400,1401,1402,1403,1404,1405,#1406
   ##1500,1501,1502,1503,1504,1505,1506,1507,1508,1509,1510,1511,1512,1513,
   ##1600,
   ##1700,
   ##1800
   ]);
if debug: print("allHabsList:",str(allHabsList.getInfo()))

Export bounds:  {'geodesic': False, 'type': 'Polygon', 'coordinates': [[[-180, -90], [180, -90], [180, 90], [-180, 90], [-180, -90]]]}
allHabsList: [100, 200, 300, 400, 500, 600, 800, 900, 1000, 1100, 1200, 1400]





#Functions
Various functions - some from functions in module "nmap" `("users/corinnaravilious/UNEP-WCMC_SharedScripts:modules_shared/nmap.js")`

In [456]:
#####habitat map refinement functions
def habitatsStringsToNumbers(habitatsList):
  return ee.List(habitatsList).map(lambda element: ee.Number.parse(element)).distinct()

#refines by selecting habitats from a discrete/categorical habitat image based on the input habitat list, 
#and then clips the resulting boolean image by the footpring of a feature collection - typically all features for a given "id_no" and "seasonal" combination
def refineByHabitatImage (featureCollection,habMapImage,habitatsList):
  return (habMapImage.updateMask(habMapImage.eq(ee.Image.constant(habitatsList)).reduce(ee.Reducer.anyNonZero())).gt(0)).unmask().clipToCollection(featureCollection)

#refines by selecting fractional images for different classes in the habitat map based on the input habitat list, 
#and then clips the resulting fractional image by the footpring of a feature collection - typically all features for a given "id_no" and "seasonal" combination
def refineByHabitatImageCollection (featureCollection,habMapImageCollection,habitatsList):
  return (ee.Image(habMapImageCollection.filter(ee.Filter.inList(habMapProperty,habitatsList)).sum()).toUint16()).unmask().clipToCollection(featureCollection)

def addPrefixToListElements (inList,newPrefix):#note this also makes the list distinct
  inList2 = [str(x) for x in inList.getInfo()]
  return ee.List(inList2).map(lambda element: ee.String(ee.String(newPrefix).cat(ee.String(element)))).distinct()

#refines by habitat based on "habMapFormat": "image" (categorical) or "imageColl" (fractional/proportional habitat classes)
# and if "refineByHabitatOption" is "noHabRefine" or if habitats are not mappable then uses backupImage based on all habitats
#TO ADD defaults to "image" format if "habMapFormat" not provided
def refineByHabitat (featureCollection,refineByHabitatOption,habMapFormat,habMap,habitatsList,backupImageCreationOption):
  habsPresentInHabMapList = checkHabsPresentInHabMapList(habitatsList,allHabsList)##test to see if present and mappable - returns True or False
  if debug: print ("refineByHabitatOption = ",refineByHabitatOption)
  if debug: print ("habsPresentInHabMap = ",habsPresentInHabMapList)
 
  if refineByHabitatOption == "noHabRefine" or habsPresentInHabMapList==False:
    if debug: print ("NOT REFINING BY HABITATS")
    if "backupImage" in globals():
      del backupImage#remove any old ones

    if backupImageCreationOption == "fromList":
     backupImage = refineByHabitatFlexibleFormat (featureCollection,habMapFormat,habMap,backupHabsList)
     return backupImage

    elif backupImageCreationOption == "fromImage":
      backupImage = ee.ImageCollection(habMap).first().unmask().gte(0).clipToCollection(featureCollection)#make an iamge
      if habMapFormat == "image" and reprojectOutput == False:#binary output if image and not reprojected
        return backupImage
      elif habMapFormat == "imageColl" or reprojectOutput==True:#binary output image collection or reprojected
        backupImage.multiply(maxHabMapCollectionVal)
        return backupImage
       
  elif refineByHabitatOption == "habRefine":# 
    return refineByHabitatFlexibleFormat (featureCollection,habMapFormat,habMap,habitatsList)

def checkHabsPresentInHabMapList (habitatsList,allHabsList): 
  return listContainsTest(habitatsList.getInfo(),allHabsList.getInfo())#test to see if present and mappable

#refines by habitat based on "habMapFormat": "image" (categorical) or "imageColl" (fractional/proportional)
#TO ADD defaults to "image" format if "habMapFormat" not provided
def refineByHabitatFlexibleFormat (featureCollection,habMapFormat,habMap,habitatsList):
  if habMapFormat == "imageColl":
    return refineByHabitatImageCollection(featureCollection,habMap,addPrefixToListElements(habitatsList,imagePrefixInHabMapImageColl,).getInfo())
  elif habMapFormat == "image": 
    return refineByHabitatImage(featureCollection,habMap,habitatsList)


###elevation refinement functions
#refines by elevation limits based on "elevFormat": "image" (categorical) or "imageColl" (fractional/proportional)
#defaults to "image" format if "elevFormat" not provided 
#TO DO split into seperate functions
def refineByElevation (image,refineByElevationOption,elevFormat,dem,minVal=-99999,maxVal=99999):
  if debug: print ("refineByElevationOption = ",refineByElevationOption)
  if refineByElevationOption == "elevRefine": 
    if debug: print ("refining by elevation")
    if minVal>=maxVal:
      if debug: print ("min elevation val larger than max elevation - no refinement")
      minVal=-99999
      maxVal=99999
    if debug: print ("elevFormat = ",elevFormat, elevFormat)
    if elevFormat == "image":
      return refineByElevationImage(image,dem,minVal,maxVal)
    elif elevFormat == "imageColl":
      return refineByElevationImageCollection(image,dem,minVal,maxVal)
  elif refineByElevationOption == "noElevRefine":
    if debug: print ("not refining by elevation")
    return image

# Function to refine AOH by image collection of elevation slices- ideal for quick low res (e.g. 10km) refinement
# Filters dem slice IC by species' limits, sums proportions and multiplies AOH by it - thus the image outside of limits becomes 0 
def refineByElevationImageCollection(image,demSliceCollection,minVal,maxVal): 
  demSliceSelectSum = demSliceCollection.filter(ee.Filter.And(ee.Filter.lt("gtMinVal",maxVal),ee.Filter.gt("lteMaxVal",minVal))).sum()
  speciesDEM = image.multiply(demSliceSelectSum).divide(ee.Number(maxElevCollectionVal)) #divide constant due to imageCollection being up to 10000
  return speciesDEM

#refines a boolean or fractional input image by the elevation ranges for the species - sets areas outside of range to 0
def refineByElevationImage (image,demImage,minVal,maxVal):#CHECK IF THIS DOES WHAT IT SHOULD
    return image.where(image.gt(0).And(dem.gte(ee.Number(minVal))).And(dem.lte(ee.Number(maxVal))),image)


##converting habitat codes/values
##TO DO = make a function to convert recode input land covers to the iucn habitat types

#for use with lvl1WithLvl2Exceptions option - input ee.List() of lvl1 habitats to recode, and the lvl2toLvl1ExceptionsList to add to new list, exports and ee.List of recoded habitats - including 
def recodeLevel1toLevel2(habitatsList,LUTlvl1toLvl2,lvl2toLvl1ExceptionsList):
  lvl2toLvl1ExceptionsList = lvl2toLvl1ExceptionsList.getInfo()#using regular list as easier 
  habitatsList = habitatsList.getInfo()#using regular python lists as easier   
  lvl2ExceptionsMatching = (list(set(habitatsList) & set(lvl2toLvl1ExceptionsList)))#select only those in both lists
  selectedHabitats = LUTlvl1toLvl2[LUTlvl1toLvl2['level1'].isin(habitatsList)]#selecting from dataframe using habitatsList
  habitatsListLevel2 = ee.List(list(selectedHabitats["level2"]))# making back into an ee.List()
  
  recodedHabitatsListLevel2InclExceptions = ((habitatsListLevel2).cat(lvl2ExceptionsMatching))
  return recodedHabitatsListLevel2InclExceptions # exporting an ee.List

#make a lookup table
def createLUTLvl1toLvl2(allHabsList,lvl2toLvl1ExceptionsList):
    level2ExceptionsRemoved = allHabsList.removeAll(lvl2toLvl1ExceptionsList)
    level1ExceptionsRemoved = ee.List(lvl2HabsToLvl1IntegerInput(level2ExceptionsRemoved))
    df = pd.DataFrame(list(zip(level2ExceptionsRemoved.getInfo(),level1ExceptionsRemoved.getInfo())),columns =['level2', 'level1'])
    return df

# Converts a list of lvl2 IUCN habitat codes to level 1 IUCN habitat code and removes duplicates - except those in the lvl2ExceptionsList which stay as they are
def lvl2HabsToLvl1WithExceptions (habsList,lvl2toLvl1ExceptionsList):#converts ee.List() of level 2 to level 1 habitats (based on M Jung's habitat map classes)
  habsListExceptionsRemoved = habsList.removeAll(lvl2toLvl1ExceptionsList)#remove level 2 habs in exceptions list
  lvl1habs = lvl2HabsToLvl1(habsListExceptionsRemoved)
  outputHabs = lvl1habs.cat(lvl2toLvl1ExceptionsList)#exceptions from level 2 added to those converted to level 1
  return outputHabs

# Converts a list of lvl2 IUCN habitat codes to level 1 
def lvl2HabsToLvl1(habsList):
    firstElementInList = habsList.get(0)
    if stringTest(firstElementInList):
      outHabs = lvl2HabsToLvl1StringInput(habsList).distinct()
    elif integerTest(firstElementInList):
      outHabs = lvl2HabsToLvl1IntegerInput(habsList).distinct()
    else:
      outHabs = print("Incorrect input type: should be an ee.List of strings or numbers")
    return outHabs

# Converts a list of lvl2 IUCN habitat codes to level 1 IUCN habitat code and removes duplicates
def lvl2HabsToLvl1StringInput (habsList):#converts ee.List() of level 2 to level 1 habitats (based on M Jung's habitat map classes)
  lvl1habs = (habsList.map(lambda element:ee.Number.parse(ee.String(element)).divide(100).floor().multiply(100)))#.distinct())
  return lvl1habs

# Converts a list of lvl2 IUCN habitat codes to level 1 IUCN habitat code and removes duplicates
def lvl2HabsToLvl1IntegerInput (habsList):#converts ee.List() of level 2 to level 1 habitats (based on M Jung's habitat map classes)
  lvl1habs = (habsList.map(lambda element:ee.Number(element).divide(100).floor().multiply(100)) )#.distinct())
  return lvl1habs


##logical tests for input type - NB for eeobjects else should throw an error

##TO DO
## add a test for image and image collection type 
##could limit the questions at the start and rely on this test mixed with "runScenarios" option

def stringTest(eeObject):
  if ee.Algorithms.ObjectType(eeObject).getInfo()== "String":
    return True

def integerTest(eeObject):
  if ee.Algorithms.ObjectType(eeObject).getInfo()== "Integer":
    return True

def listTest(eeObject):
  if ee.Algorithms.ObjectType(eeObject).getInfo()== "List":
    return True


def imageNames (imageCollection):##list existing images in collection (if any)
  return imageCollection.aggregate_array("system:id").getInfo()

# Returns the intersection of two lists #<CHECK THIS IS NEEDED>
def listContainsTest(List1, List2): 
    set1 = set(List1) 
    set2 = set(List2) 
    if set1.intersection(set2): 
        return True 
    else: 
        return False

##general functions for attributes in a collection
def collectionPropertyNames(collection):#gets fields as a list from a collection
  return collection.first().propertyNames().getInfo()

def collectionPropertiesToDf(collection,propertySelection=None):#attribute table into pandas dataframe - slower than geemap version, but can do >5000 records
  list()
  i = 0
  if propertySelection==None:
    collectionPropertieslist = collection.first().propertyNames().getInfo()
  else:
    collectionPropertieslist = propertySelection 
  nestedList = list()
  for property in collectionPropertieslist:
    nestedList.append(collection.aggregate_array(property).getInfo())
  df = pd.DataFrame(data=nestedList).transpose()
  df.columns = collectionPropertieslist
  return df

##filtering inputs

def filteringIUCNByPresenceOriginSeasonCategory (IUCNpolys,applyIUCNfiltering):
  if (applyIUCNfiltering)==True: 
    print ("using IUCN filter")
    IUCNpolys_P = IUCNpolys.filter(ee.Filter.inList(presenceField, presenceValsList));
    IUCNpolys_PO = IUCNpolys_P.filter(ee.Filter.inList(originField, originValsList));
    IUCNpolys_POS = IUCNpolys_PO.filter(ee.Filter.inList(seasonField, seasonalValsList));
    IUCNpolys_POS_category = IUCNpolys_POS.filter(ee.Filter.inList(categoryField, categoryValsList));

    #some of the below could be shortened
    #print ("LUT feature collection size",str((lut.aggregate_array(lutSpeciesId).distinct()).distinct().size().getInfo()))
    print ("feature collection size before filtering (IUCN DATA)",str((IUCNpolys.aggregate_array(speciesId).distinct()).distinct().size().getInfo()))
    print ("feature collection size after filtering (IUCN DATA - origin)",str((IUCNpolys_PO.aggregate_array(speciesId).distinct()).size().getInfo())," removed: ",((IUCNpolys_P.aggregate_array(speciesId).distinct()).size().subtract((IUCNpolys_PO.aggregate_array(speciesId).distinct()).size()).getInfo()))
    print ("feature collection size after filtering (IUCN DATA - seasonal)",str((IUCNpolys_POS.aggregate_array(speciesId).distinct()).size().getInfo())," removed: ",((IUCNpolys_PO.aggregate_array(speciesId).distinct()).size().subtract((IUCNpolys_POS.aggregate_array(speciesId).distinct()).size()).getInfo()))##expecting 0 removed here as i think this is all seasons
    print ("feature collection size after filtering (IUCN DATA - category)",str((IUCNpolys_POS_category .aggregate_array(speciesId).distinct()).size().getInfo())," removed: ",((IUCNpolys_POS_category .aggregate_array(speciesId).distinct()).size().subtract((IUCNpolys_POS_category.aggregate_array(speciesId).distinct()).size()).getInfo()))##expecting 0 removed here as i think this is all seasons
    print ("count of input id_no (duplicates removed) before filtering: ",str(IUCNpolys.aggregate_array(speciesId).distinct().length().getInfo()), "and after filtering: ",str(IUCNpolys_POS_category.aggregate_array(speciesId).distinct().length().getInfo()))
    print ("count of id_no - i.e. species removed by filtering: ",str(IUCNpolys.aggregate_array(speciesId).distinct().length().subtract(IUCNpolys_POS_category.aggregate_array(speciesId).distinct().length()).getInfo()))  
    print ('Categories of input features (pre-filtering) %s loaded %s ' % ( tax, str(IUCNpolys_POS_category.aggregate_array(categoryField).distinct().getInfo()) ) ) 

    return IUCNpolys_POS_category
  else:
    if debug: print ("not filtering IUCN polys")

###various functions for finding max extent of input features - useful for setting export extent
#cant recall if the extent fucntions are implemented in code as a lot of junk in main part currently

def setTotalFcExtentFromProperties(fc):
  xminForFC = fc.aggregate_array("xmin").reduce(ee.Reducer.min())
  xmaxForFC = fc.aggregate_array("xmax").reduce(ee.Reducer.max())
  yminForFC = fc.aggregate_array("ymin").reduce(ee.Reducer.min())
  ymaxForFC = fc.aggregate_array("ymax").reduce(ee.Reducer.max())
  return fc.set({
  "xminForFC":xminForFC,
  "xmaxForFC":xmaxForFC,
  "yminForFC":yminForFC,
  "ymaxForFC":ymaxForFC,
  })

def setTotalFcExtentFromGeometries(fc):
  fc = setExtentPropertiesForEveryFeature(fc)
  xminForFC = fc.aggregate_array("xmin").reduce(ee.Reducer.min())
  xmaxForFC = fc.aggregate_array("xmax").reduce(ee.Reducer.max())
  yminForFC = fc.aggregate_array("ymin").reduce(ee.Reducer.min())
  ymaxForFC = fc.aggregate_array("ymax").reduce(ee.Reducer.max())
  return fc.set({
  "xminForFC":xminForFC,
  "xmaxForFC":xmaxForFC,
  "yminForFC":yminForFC,
  "ymaxForFC":ymaxForFC,
  })  

def setExtentPropertiesForEveryFeature(fc):
  return fc.map(setExtentPropertiesForFeature)

def setExtentPropertiesForFeature(feature):
  listCoord = ee.List(feature.geometry().bounds().coordinates().flatten()) #CHECK IF GEOMETRY CALL CAUSES ERRORS
  return feature.set({
  "xmin":ee.Number(listCoord.get(0)),
  "ymin":ee.Number(listCoord.get(1)),
  "xmax":ee.Number(listCoord.get(4)),
  "ymax":ee.Number(listCoord.get(5)),
  })

#reprojecting images

# Takes an input image and reprojects it in resolution and projection to a provided template image
def reprojectImage (image,template_image,reducer):
  targProj = template_image.projection() 
  inProj = image.projection()
  image = image.reproject(crs = inProj)
  res = image.reproject(image.projection()).reduceResolution(reducer = reducer, maxPixels = 65024).reproject(crs = targProj)
  return res

# Takes an input image and reprojects it in the resolution and projection of a provided template image from an image collection
def reprojectImageMosaic (image,image_collection,template_image,reducer):##takes first image from image coll for original crs
  targProj = template_image.projection();
  inProj = ee.Image(image_collection.first()).projection()
  image = image.reproject(crs= inProj)
  res = image.reproject(image.projection()).reduceResolution(reducer = reducer, maxPixels= 65024).reproject(crs = targProj)
  return res
  



# Importing assets 1 of 5: habitat & elevation preferences


In [441]:
# Now load only the species habitat lookup table
# habitat preferences Lookup table
lookup_tables = {'lut_iucn' : 'projects/unep-wcmc/p8425_NatureMap/raw/LUT_suit_allsp_2019_2_gdb_poly_sel_taxa_level1_level2_wide_v2',
                'lut_alternative': lutAlternativeAsset}

# Load in the habitat preference data
lut = ee.FeatureCollection(lookup_tables.get(lutChoice)) 

print('Loaded in LUT lookup. Total length:',  str(lut.size().getInfo() ))


Loaded in LUT lookup. Total length: 40643


# Importing assets 2 of 5: range polygons






In [442]:
# ----------------------------------------------------------------------
# IMPORT IUCN VECTOR DATA
# Dictionary with assets and names
species_ranges = {
    # Amphibians
    'AMPHIBIA' : "projects/unep-wcmc/imported/species/iucn_rl_%s/AMPHIBIA" % (IUCN_version),
    # Mammals
    'MAMMALIA' : 'projects/unep-wcmc/imported/species/iucn_rl_%s/MAMMALIA' % (IUCN_version),
    # Birds
    'AVES_1' : "projects/unep-wcmc/imported/species/iucn_rl_%s/AVES_orderAtoC" % (IUCN_version),
    'AVES_2' : 'projects/unep-wcmc/imported/species/iucn_rl_%s/AVES_orderDtoZminusPASSERIFORMES' % (IUCN_version),
    'AVES_3' : 'projects/unep-wcmc/imported/species/iucn_rl_%s/AVES_orderPASSERIFORMES_famAtoN' % (IUCN_version),
    'AVES_4' : 'projects/unep-wcmc/imported/species/iucn_rl_%s/AVES_orderPASSERIFORMES_famOtoZ' % (IUCN_version)
                }
# ------------------------------------------- #
# Load in only the group to run
if target_group == 'AMPHIBIA':
  # IMPORT AMPHIBIANS
  fc_amp = ee.FeatureCollection(species_ranges.get('AMPHIBIA') )
  tax = 'AMPHIBIA'
  IUCNpolys = fc_amp

elif target_group == 'MAMMALIA':
  # IMPORT MAMMALS
  fc_mam = ee.FeatureCollection(species_ranges.get('MAMMALIA'))
  tax = 'MAMMALIA'
  IUCNpolys = fc_mam

elif target_group == 'AVES':
  # IMPORT BIRDS - imported in subsets due to shapefile limitations
  fc_aves_a = ee.FeatureCollection(species_ranges.get('AVES_1'))
  fc_aves_b = ee.FeatureCollection(species_ranges.get('AVES_2'))
  fc_aves_c = ee.FeatureCollection(species_ranges.get('AVES_3'))
  fc_aves_d = ee.FeatureCollection(species_ranges.get('AVES_4'))
  ##combining subsets
  fc_ave = fc_aves_a.merge(fc_aves_b).merge(fc_aves_c).merge(fc_aves_d) # CAN IMROPOVE USING nested image coll then .FLATTEN() combined birds - for all birds
  tax = 'AVES'
  IUCNpolys = fc_ave

# IUCN = fc_ave_miss

elif target_group == 'MultiTax': 
  #COMBINE ALL GROUPS INTO ONE
  IUCNpolys = ee.FeatureCollection([ee.FeatureCollection(species_ranges.get('AVES_1')),
                               ee.FeatureCollection(species_ranges.get('AVES_2')),
                               ee.FeatureCollection(species_ranges.get('AVES_3')),
                               ee.FeatureCollection(species_ranges.get('AVES_4')),
                               ee.FeatureCollection(species_ranges.get('MAMMALIA')),
                               ee.FeatureCollection(species_ranges.get('AMPHIBIA'))]).flatten()
  tax= 'MultiTax'
# ---------------------------------------------------------------------------------------- #

##checking 
print('Number of features of taxonomic group %s loaded %s ' % ( tax, str(IUCNpolys.size().getInfo()) ) ) 
print('Number of distinct species of taxonomic group %s loaded %s ' % ( tax, str(IUCNpolys.aggregate_array(speciesId).distinct().size().getInfo()) ) ) 
#<TO ADD - print number of distinct species-season combinations loaded>
print('Categories of input features (pre-filtering) %s loaded %s ' % ( tax, str(IUCNpolys.aggregate_array(categoryField).distinct().getInfo()) ) ) 


Number of features of taxonomic group MultTax loaded 51655 
Number of distinct species of taxonomic group MultTax loaded 23513 
Categories of input features (pre-filtering) MultTax loaded ['CR', 'DD', 'EN', 'EX', 'LC', 'NT', 'VU', 'EW'] 


# Importing assets 3 of 5: habitat types

In [443]:
### Environmental data: Habitat (or Land cover/use) map 
# Habitat type data and DEM for refinement
# Version 004 being https://zenodo.org/record/3666246
# Contact Martin Jung (jung - at - iiasa.ac.at) if access to a new version is required

# The main composite habitat map at level 1 and level 2 - categorical
ht_maps = {'004_1' : 'users/Uploads/habitattypes/iucn_habitatclassification_composite_lvl1_ver004',
           '004_2' : 'users/Uploads/habitattypes/iucn_habitatclassification_composite_lvl2_ver004'}

#Fractional aggregated habitat map image collections at 1km resolution
ht_maps_1kmfrac = {'004_1' : 'users/Uploads/habitattypes/Fractions_ht004_lvl1_1km',
           '004_2' :'users/Uploads/habitattypes/Fractions_ht004_lvl2_1km'
           #'004_2' : "projects/unep-wcmc/imported/habitats/jung_iucn_habMaps/Fractions_ht004_lvl2_30arcsec" #more appropriate resolution for most work - but throwing errors when used at poles
           }
#Potential Natural Vegetation images - categorical 
pnv_maps = {'004_1' : 'projects/unep-wcmc/p08880_CCI_NbS/work_in_progress/iucn_habitatclassification_composite_pnv_lvl1_ver004',
            '004_2' : 'projects/unep-wcmc/p08880_CCI_NbS/work_in_progress/iucn_habitatclassification_composite_pnv_lvl2_ver004'}

#Fractional aggregated PNV image collections at 1km resolution
#pnv image col - NB level 2 is likely too fine - need to check?
# pnv_maps_1kmfrac = {'004_1' : TBC,
#             '004_2' : TBC}

# Load the habitat types map
if ('%s_1' % (ht_version)) in ht_maps:
  if habMapFormat=="image":
    if habMapInputChoice == "habMap":
      lvl1_image = ee.Image(ht_maps.get('%s_%s' % (ht_version, '1')) ) # Load the composite map at level 1
      lvl2_image = ee.Image(ht_maps.get('%s_%s' % (ht_version, '2')) ) # Load the composite map at level 1
      print('Loaded lvl1  habitat type layer %s at %s m resolution' % (ht_version, lvl1_image.projection().nominalScale().getInfo()) )
      print('Loaded lvl2 habitat type layer   %s at %s m resolution' % (ht_version, lvl2_image.projection().nominalScale().getInfo()) )
    elif habMapInputChoice == "potentialNaturalVegetation":
      lvl1_image = ee.Image(pnv_maps.get('%s_%s' % (ht_version, '1')) ) # Load the composite map at level 1
      lvl2_image = ee.Image(pnv_maps.get('%s_%s' % (ht_version, '2')) ) # Load the composite map at level 1
      print('Loaded lvl1 potential natural vegetation (PNV) layer %s at %s m resolution' % (ht_version, lvl1_image.projection().nominalScale().getInfo()) )
      print('Loaded lvl2 potential natural vegetation (PNV)  layer   %s at %s m resolution' % (ht_version, lvl2_image.projection().nominalScale().getInfo()) )
    elif habMapInputChoice == "altHabMap":
      print("alternative altHabMap")
      if runScenarios ==True:
        lvl1_image = ee.ImageCollection(altHabMap_image)###may need to rethink
        lvl2_image = ee.ImageCollection(altHabMap_image)###may need to rethink
      if runScenarios ==False:
        lvl1_image = ee.Image(altHabMap_image)###may need to rethink
        lvl2_image = ee.Image(altHabMap_image)###may need to rethink

  if habMapFormat=="imageColl":
    if habMapInputChoice == "habMap":
      lvl1_1kmColl = ee.ImageCollection(ht_maps_1kmfrac.get('%s_%s' % (ht_version, '1')))
      lvl2_1kmColl = ee.ImageCollection(ht_maps_1kmfrac.get('%s_%s' % (ht_version, '2')))
      print('Loaded lvl1 habitat type image collection %s at %s m resolution' % (ht_version, lvl1_1kmColl.first().projection().nominalScale().getInfo()) )
      print('Loaded lvl2 habitat type image collection %s at %s m resolution' % (ht_version, lvl2_1kmColl.first().projection().nominalScale().getInfo()) )
    elif habMapInputChoice == "potentialNaturalVegetation":
      lvl1_1kmColl = ee.ImageCollection(pnv_1kmfrac.get('%s_%s' % (ht_version, '1')))
      lvl2_1kmColl = ee.ImageCollection(pnv_maps_1kmfrac.get('%s_%s' % (ht_version, '2')))
      print('Loaded lvl1 potential natural vegetation (PNV) image collection %s at %s m resolution' % (ht_version, lvl1_1kmColl.first().projection().nominalScale().getInfo()) )
      print('Loaded lvl2 potential natural vegetation (PNV) image collection %s at %s m resolution' % (ht_version, lvl2_1kmColl.first().projection().nominalScale().getInfo()) )
    elif habMapInputChoice == "altHabMap":
      lvl1_1kmColl = ee.ImageCollection(altHabMap_1kmfrac)###may need to rethink
      lvl2_1kmColl = ee.ImageCollection(altHabMap_1kmfrac)###may need to rethink
      print("alternative altHabMap")
  else:
    print('map not found in shared assets. Check version numbers!')
print("---")

# lvl1_image = ee.Image("users/Uploads/habitattypes/iucn_habitatclassification_composite_lvl1_ver004");
# lvl2_image = ee.Image("users/Uploads/habitattypes/iucn_habitatclassification_composite_lvl2_ver004");
# lvl2_1kmColl =  ee.ImageCollection("users/Uploads/habitattypes/Fractions_ht004_lvl2_1km");
# lvl1_1kmColl =  ee.ImageCollection("users/Uploads/habitattypes/Fractions_ht004_lvl1_1km");

Loaded lvl1 potential natural vegetation (PNV) layer 004 at 100 m resolution
Loaded lvl2 potential natural vegetation (PNV)  layer   004 at 100 m resolution
map not found in shared assets. Check version numbers!
---


#Importing assets 4 of 5: elevation data

In [444]:
### Environmental data: DEM
# Now load the DEM - Combined from gmted with others to get global coverage
demMaps = {"demImage250m" : "projects/unep-wcmc/imported/topography/DEMs/dem_composite",    
           "demImage1km" :"projects/unep-wcmc/imported/topography/DEMs/dem_composite_agg_30arcsec",# "projects/unep-wcmc/imported/topography/DEMs/dem_composite_agg1km", 
           "demImageCollection1km" : "", 
           "demImageCollection10km": ""}
           
#Load the elevation choice
try:
  if elevFormat=="image":
    dem = ee.Image(demMaps.get(demMapChoice))
    print('Loaded DEM image at %s m resolution' % (dem.projection().nominalScale().getInfo()) )
  elif elevFormat=="imageColl":
    dem = ee.ImageCollection(demMaps.get(demMapChoice))
    print('Loaded DEM iamge collection at %s m resolution' % (dem.projection().nominalScale().getInfo()) )
except:
  print('DEM not found in shared assets. Check input assets!')
      
print("---")
#-------------------------------------------------------------


Loaded DEM image at 231.91560581931986 m resolution
---


#Importing assets 5 of 5: template images

In [445]:
###NB NEED BETTER GRIDS - thse done align with other wgs84 as specific to NatureMap
# Global template grids. Reprojected from default Nature Map template grid
# from M Jung - mollweide versions didnt work with code
# TODO: In the future best define and export a template grid directly in Earth Engine
if reprojectOutput:
  templates = {#"templateImage1km": "projects/unep-wcmc/imported/grids/squares/globalgrid_wgs84_1km",
               "templateImage1km": "Oxford/MAP/accessibility_to_healthcare_2019",##extent issues - seems to be at poles
              "templateImage10km": "projects/unep-wcmc/imported/grids/squares/globalgrid_wgs84_10km" }
  templateImage = ee.Image(templates.get(templateImageChoice))
  print('Loaded template image at %s m resolution' % (templateImage.projection().nominalScale().getInfo()) )

Loaded template image at 927.6624232772797 m resolution


# Filtering data
Filter the IUCN provided range maps using standard criteria, e.g. Presence, origin and season (according to IUCN guidelines in email commmunication with Andy Arnell june 2018)(PRESENCE = 1 or PRESENCE = 4) AND (origin = 1 or ORIGIN  = 2) AND (SEASONAL = 1 or SEASONAL = 2 OR SEASONAL = 3 OR SEASONAL = 4 OR SEASONAL = 5)    

In [446]:
###NB - may want to change the filters in parameters e.g. extinct range for species, when thinking of past species range data
# For all data from IUCN
fc = filteringIUCNByPresenceOriginSeasonCategory (IUCNpolys = IUCNpolys,applyIUCNfiltering=applyIUCNfiltering)


using IUCN filter
feature collection size before filtering (IUCN DATA) 23513
feature collection size after filtering (IUCN DATA - origin) 23496  removed:  3
feature collection size after filtering (IUCN DATA - seasonal) 23496  removed:  0
feature collection size after filtering (IUCN DATA - category) 7084  removed:  0
count of input id_no (duplicates removed) before filtering:  23513 and after filtering:  7084
count of id_no - i.e. species removed by filtering:  16429
Categories of input features (pre-filtering) MultTax loaded ['CR', 'DD', 'EN', 'VU'] 


## Extra filtering
Only if doing extra filtering - e.g. for species in an area of interest (AOI) or for rerunning empty images with different refinement (e.g. by level 1 habitats, or non-anthropogenic habitats). 


In [447]:
if filterTasksBySpeciesList:
  # output format: a list of distinct ids (i.e. choose either speciesId or idSeasonCombined variables from parameters) 
 # need to set filterTasksBySpeciesList to true
 # then in the below make the speciesKeepList variable = <yourlist> (make it a client side object - i.e. get info and remove duplicates)
 #add featureList variable creation code here:
 speciesKeepList = [22695207]
 #######################################################



#Export region preparation
Setting XY values for the bounds of each feature as porperties (for creating export region)

In [448]:
if staticExportRegion ==False:
  fc = setExtentPropertiesForEveryFeature(fc)
  print (str(fc.aggregate_array("xmin").getInfo()))


[-17.533778366754788, -1.2070251682871782, 21.543914905247906, -15.540444272291621, 1.711585772336658, 21.872867195073084, -17.533778366754788, -16.520501223091312, -45.93592890003111, -55.227620880280334, '-Infinity', -0.24105899229868635, -17.532137441742673, 5.382647771626678, 3.769719603505753, 6.432453961850404, 6.863751560335529, 7.236153377424937, -15.504374620621153, -54.53387395259179, -55.64270075667789, 8.783323218627341, 9.258913080897939, -41.924802557346815, 6.470276017758223, -69.29134912179826, -38.45168899518171, 6.563475669697787, 6.517515755841412, 6.502684756845411, 6.486877293679975, 6.517698538690173, -39.99627701265925, -37.38177675439002, -16.789028246475983, -15.487768966417756, -17.54318713370836, -14.388743209395793, -16.51538210794382, -5.772582771275313, -46.442505102399075, -17.532779545555872, -5.015271387051524, -85.4359124581449, -49.21862993752427, -17.535218687420564, -11.812502068381592, -58.80511305080916, -84.83551231473203, -18.16961499846168, -17

# Main processing: step 1
Preparing habitat maps


In [449]:

# sorting out habitat maps to use depending on if level 1 or level2, and image or fractional image collection
##TO DO put into a function???? 

if habMapFormat == "imageColl":
  if habMapLevel == "level1":
    # Load all habitat preferences 
    if debug: print("using lvl 1 hab map")
    habMap = lvl1_1kmColl
    ##allHabsList = lvl1_1kmColl.aggregate_array("system:index").distinct()
  elif habMapLevel == "level2":
    if debug: print("using lvl 2 hab map")
    habMap = lvl2_1kmColl
    ##allHabsList = lvl2_1kmColl.aggregate_array("system:index").distinct()
  elif habMapLevel  == "lvl1WithLvl2Exceptions":
    if debug: print("using lvl 2 hab map")
    habMap = lvl2_1kmColl
elif habMapFormat == "image":
  if habMapLevel == "level1":
    habMap = lvl1_image
  elif habMapLevel == "level2":
    habMap = lvl2_image
  elif habMapLevel  == "lvl1WithLvl2Exceptions":
    if debug: print("using lvl 2 hab map")
    habMap = lvl2_image

# print (habMapScenarioList)
# print(habMap.get("system:index").getInfo())

##Getting output scale from inputs for use in reprojections and exporting 
##TO DO put into a function
if habMapFormat == "imageColl" or runScenarios == True:
  output_scale = habMap.first().projection().nominalScale().getInfo()
else:
  output_scale = habMap.projection().nominalScale().getInfo()
output_scale_int = ee.Number(output_scale).int().getInfo()##integer version for descriptions in exporting
print ("output scale chosen: ",str(output_scale))
print ("output scale integer: ",str(output_scale_int))

if "multiband" in globals():#delete variable to clear any remaining bands from previous failed/aborted runs 
  del (multiband)
if "image" in globals():
  del (image)

#getting backupList of habitats 
#TO DO - make lis as an asset?
##also if in a function then could have a backup list of user choice and have an alternative - say all non-anthropogenic
#TO DO - think about if clipped region - hard to get new list that is accurate - 
#TO DO - Related to this, need to get a "try" statement in the refineByHabitat function for when error thrown from no inputs (the "except" part will be to use backup image replacement
#as a function to make cleaner
if backupImageCreationOption == "fromList": 
  backupHabsList = allHabsList
  print ("backupHabsList ",str(backupHabsList.getInfo()))

  if len(allHabsList.getInfo()) == 0:
    print ("WARNING 'allHabsList' empty - refinement by missing habitats will throw error")

##TO ADD -CLIPPING OPTION FOR HABMAP

output scale chosen:  100
output scale integer:  100


# Main processing: step 2
Preparing lists of tasks to run

In [450]:
###Setting up tasks list based on "id_no" and "seasonal" values from input features 
#get list of species-season combinations to run
# print ("number input features - incl duplicates id_no-season combinations: ",str(len(idListAll)))##check list
data = collectionPropertiesToDf(fc,[speciesId,seasonCode])
data[idSeasonCombined] = data[speciesId].map(str)+"_"+data[seasonCode].map(str)#add field for concatonated string of id_no and seasonal values
data = data.drop_duplicates()### FIX? ###removes duplicate rows

print ("before filterTasksBySpeciesList applied - showing top ten: ")
print ( data.head())

#to do make into a function
if filterTasksBySpeciesList == True:
  if filterTasksField == "speciesId":
    data = data[data[speciesId].isin(speciesKeepList)]# if id_no used to filter
    print ("filtering tasks by list using: ",str(speciesId))
  elif filterTasksField == "idSeasonCombined":
    data = data[data[idSeasonCombined].isin(speciesKeepList)] # if id_no-season concatonated used to filter
    print ("filtering tasks by list using speciesKeepList and: ",str(idSeasonCombined))
  else:
    print ("no id column listed")

print ("after filterTasksBySpeciesList applied - showing top ten: ")
print (data.head())

idList = list(data[speciesId])#make speciesId field from dataframe into a list 
idList2 = list(data[seasonCode])#make seasonCode field from dataframe into a list 
print ("number distinct inputs - distinct id_no-season combinations: ",str(len(idList)))

##limiting number of species-seasons to process based on input parameters for startNumber and maxExportNumber 
listLength = len(idList[:startNumber+exportMaxNumber])

worklist = range(startNumber,listLength)#main species-seasons worklist to loop through
print ("listLength: ",str(listLength))
print ("startNumber: ",str(startNumber))
print ("exportMaxNumber: ",str(exportMaxNumber))
print ("numberOfBandsPerImage: ",str(numberOfBandsPerImage))
print ("worklist length",str(len(worklist)))


before filterTasksBySpeciesList applied - showing top ten: 
       id_no  seasonal     id_seas
0   22695207         1  22695207_1
6   22695250         1  22695250_1
8   22690819         1  22690819_1
10  22695185         1  22695185_1
11  22695189         3  22695189_3
filtering tasks by list using:  id_no
after filterTasksBySpeciesList applied - showing top ten: 
       id_no  seasonal     id_seas
0   22695207         1  22695207_1
62  22695207         3  22695207_3
number distinct inputs - distinct id_no-season combinations:  2
listLength:  2
startNumber:  0
exportMaxNumber:  100000
numberOfBandsPerImage:  50
worklist length 2


#Main processing: step 3
Start the main processing going over all features in the list of tasks and exporting them by numberOfBandsPerImage

In [451]:
print (habMapScenarioList)
print(habMap.get("system:index").getInfo())
for scenario in habMapScenarioList:
  if runScenarios == True:
    habMap = habMap.filter(ee.Filter.stringContains("system:index",scenario))
    if habMapFormat == "image":
      habMap = habMap.first()
  
  #running main analyses
  #Set up a progress bar
  pbar = tqdm(total = len(worklist))
  if 'multiband' in globals(): del multiband
  if 'image' in globals(): del image
  if 'xminForMultiband' in globals(): del xminForMultiband 
  if 'xmaxForMultiband' in globals(): del xmaxForMultiband 
  if 'yminForMultiband' in globals(): del yminForMultiband 
  if 'ymaxForMultiband' in globals(): del ymaxForMultiband 
  ##-----------------------------------------------------------------------------------------
  #loop over species list in batches. i = batch counter, f = image creation counter

  for i in tnrange(0, len(worklist), numberOfBandsPerImage,desc = 'Overall progress'):
    batch = worklist[i:i+numberOfBandsPerImage] #making sets of batches to run through based on input numberOfBandsPerImage parameter - note: the result might be shorter than numberOfBandsPerImage for the last run
    if debug: print ("i",str(i)) #feedback to console
    if debug: print ("batch",str(batch)) #feedback to console
    # Update batch description
    pbar.set_description('Batch %i' %i)

    for f in batch:
      #for getting extents of export
      if 'xminForFC' in globals(): del xminForFC
      if 'xmaxForFC' in globals(): del xmaxForFC
      if 'yminForFC' in globals(): del yminForFC
      if 'ymaxForFC' in globals(): del ymaxForFC

      if debug: print ("f",str(f)) #feedback to console
      id = idList[f]#get id_no value for this loop
      id2 = idList2[f]##get seasonal value for this loop
      
      if debug: print ("Processing feature no: ",str(f-startNumber+1)) #feedback to console
      fc1 = ee.FeatureCollection(fc.filter(ee.Filter.And((ee.Filter.eq(speciesId,id),ee.Filter.eq(seasonCode,id2)))))#select spratial features by "id_no" and "seasonal" values
      print("here")
      #selecting first of each feature collection for a) getting use in scripts below
      firstSpatial = fc1.first()#get first one of the selected features (as typically more than one feature per sepcies) 

      lutSel = ee.FeatureCollection(lut.filter(ee.Filter.And((ee.Filter.eq(lutSpeciesId,id),ee.Filter.eq(lutSeasonCode,id2)))))#select LUT vals by "id_no" and "seasonal" values
      firstLutSel = ee.Feature(lutSel.first())#get first row/feature from the selected features
      
      if debug: print ("first feature habitats",firstLutSel.get(lutHabField).getInfo())##get info for first row/feature from LUT 
  ##------------------------------------------------------------------------------------------------------------------------------------
  
  
    #GETTING LIST OF HABITATS FOR FEATURE BEING PROCESSED - MESSY, SO MAKE INTO FUNCTION(S) IDEALLY
    #TO DO deprecate WIDE format soon
      if refineByHabitatOption == "habRefine":
        if debug: print ("chose to refine by habitats")
        try:
          if lutFormat == "long":
            print ("Using long format LUT")
            habitatsList = lutSel.aggregate_array(lutHabField).distinct()
            habitatsListLvl1 = lvl2HabsToLvl1(habitatsList).distinct()
            habitatsList = ee.List([habitatsList,habitatsListLvl1]).flatten().distinct()
          elif lutFormat == "wide": #(i.e. a list stored as a text delimited string)
            print ("Using wide format LUT")
            inVal=(ee.String((firstLutSel.get(lutHabField))))
            # habitatsList = habitatsList.map(lambda element: ee.Number.parse(element))
            print ("inVal", str(inVal.getInfo()))
            habitatsList = inVal.split(delimiter).distinct().replace("NA","0").replace("N","0") #get string from field, convert to list using ";" seperator and replace NAs with 0
            print ("conc str list",str(habitatsList.getInfo()))
            ###habitatsList = habitatsList.map(lambda element: ee.Number.parse(element)).distinct()# if using integers from deprecated htclass property for example
            #habitatsList = habitatsList.map(lambda element: ee.String(ee.String(imagePrefixInHabMapImageColl).cat(ee.String(element)))).distinct()#if using system:index sting with the htclass prefix
          print (str(habitatsList.getInfo())) #useful for error catching <need to replace with something else for non-verbose runs>
        except:
          if backupImageCreationOption == "fromList":
            if debug: print ("WARNING no LUT values - so refining by backupHabsList")
            habitatsList = backupHabsList
          if backupImageCreationOption == "fromImage":
            habitatsList = ["0"]
      elif refineByHabitatOption == "noHabRefine":
        if backupImageCreationOption == "fromList":
          if debug: print ("chose to refine by backup habitat list - e.g. either all habitats, or all but anthropgenic habitats")
          habitatsList = backupHabsList


      ##level 1 refinement option - TO DO make into function - 2 inputs habitatsList and habmaplevel
      if habMapLevel=="level1":
        if debug: print (habMapLevel,"recoding habitat levels")  
        habitatsList = lvl2HabsToLvl1(habitatsList)
      elif habMapLevel=="level2":
        if debug: print (habMapLevel,"habitat codes") 
        habitatsList = habitatsList
      elif habMapLevel=="lvl1WithLvl2Exceptions":
        if debug: print (habMapLevel,"- recoding habitat levels") 
        LUTlvl1toLvl2 = createLUTLvl1toLvl2(allHabsList,lvl2toLvl1ExceptionsList) #automatically makes a lookup table based on input paramters (i.e., allHabsList and exceptions list)
        lvl1HabitatsList = lvl2HabsToLvl1WithExceptions(habitatsList,lvl2toLvl1ExceptionsList)
        recodedValuescombined = recodeLevel1toLevel2(lvl1HabitatsList,LUTlvl1toLvl2,lvl2toLvl1ExceptionsList)
        habitatsList = recodedValuescombined
              
      if debug: print ("habitats list", str(habitatsList.getInfo()))
    ###---------------------------------------------------------------
      ##GETTING ELEVATION REFINEMENT LIMITS
      if refineByElevationOption == "elevRefine":
        minVal = firstLutSel.get(minValField).getInfo()
        maxVal = firstLutSel.get(maxValField).getInfo()
        if debug: print ("max elev",str(maxVal))
        if debug: print ("min elev",str(minVal))
  ##---------------------------------------------------------------------------------------
  #------------------------------------------------------------
      ##RUNNING REFINEMENT
      image = refineByHabitat(refineByHabitatOption = refineByHabitatOption, \
                              habMapFormat = habMapFormat,\
                              featureCollection = fc1,\
                              habMap = habMap,\
                              habitatsList = habitatsList,\
                              backupImageCreationOption = backupImageCreationOption\
                              ).rename("constant")

      image = refineByElevation(image=image,
                                  refineByElevationOption = refineByElevationOption,\
                                  elevFormat = elevFormat,\
                                  dem = dem,\
                                  minVal = minVal,\
                                  maxVal = maxVal)#.rename(bandName)
      ##-----------------------------------------------------------------------------------------------------------
      ##reprojecting

      if reprojectOutput == True:
        if habMapFormat=="imageColl":
          image = ee.Image(reprojectImageMosaic(image,habMap,templateImage,reducerChoice).round().toUint16())
        elif habMapFormat=="image":
          image = ee.Image(reprojectImage(image,templateImage,reducerChoice).multiply(maxValueInReprojectedOutput).round().toUint16())
        if debug == True: print ("aggregating to coarser resolution")

        output_scale = image.projection().nominalScale().getInfo()
        output_scale_int = ee.Number(output_scale).int().getInfo()##integer version for descriptions in exporting
        if debug == True: print ("output scale chosen: ",str(output_scale))
        if debug == True: print ("output scale integer: ",str(output_scale_int))
        #if applyMaskMultiplication: image = image.multiply(image.mask())#ADDED TO SORT ISSUES ON RANGE EDGES WHEN EXPORTING
      ##--------------------------------------------------------------------------------------------------------------------------------
      
      ##set properties for filtering through the image collection after processing
      class1 = firstSpatial.get(taxPrefix)#chose class1 as variable as "class" is a reserved word in python

      category1 = firstSpatial.get(categoryField)
      #-----------------------------------------------------------------------------------------------------------------------------
      bandName = str(class1.getInfo())+"_"+str(category1.getInfo())+"_"+str(id)+"_"+str(id2) #getting band name string

      bandName = bandName.replace('/','')# Remove backslash if existing

      if bandName[0]=="_":
        bandName = "noClassDefined"+str(bandName)

      image = image.rename(bandName)##properties dont appear to be translated into indiviudal bands in multiband output, so putting in name as can be searched using stringContains later (if exporting to GEE asset that is)     
    
      pbar.update(f) # Update progress bar

      ptext = 'Processing feature %i : %s ' % (f-startNumber+1, str(bandName))

      pbar.set_postfix_str(s = ptext)
      ##-----------------------------------------------------------------------------------------------------------
      if staticExportRegion == False:##make into a function
        fc1 = setTotalFcExtentFromProperties(fc1)
        xminForFC = fc1.get("xminForFC").getInfo()
        xmaxForFC = fc1.get("xmaxForFC").getInfo()
        yminForFC = fc1.get("yminForFC").getInfo()
        ymaxForFC = fc1.get("ymaxForFC").getInfo()
        exportRegion = ee.Geometry.Rectangle([xminForFC, yminForFC, xmaxForFC, ymaxForFC], None, False); # for global extent
        if debug: print ("exportRegion coordinates: ", str(exportRegion.bounds().coordinates().getInfo()))

        #stacking images into bands
        if "multiband" not in globals():
          if debug: print ("multiband not in globals")
          xminForMultiband = xminForFC
          xmaxForMultiband = xmaxForFC
          yminForMultiband = yminForFC
          ymaxForMultiband = ymaxForFC
        
        else:
          if xminForMultiband > xminForFC:
            xminForMultiband = xminForFC
          if xmaxForMultiband < xmaxForFC:
            xmaxForMultiband = xmaxForFC
          if yminForMultiband > yminForFC:
            yminForMultiband = yminForFC
          if ymaxForMultiband < ymaxForFC:
              ymaxForMultiband = ymaxForFC
        # print (xminForFC)
        # print (xmaxForFC)
        # print (yminForFC)
        # print (ymaxForFC)
        # print (xminForMultiband)
        # print (xmaxForMultiband)
        # print (yminForMultiband)
        # print (ymaxForMultiband)
      ##-----------------------------------------------------------------------------------------------------------
      #first image in batch 
      if "multiband" not in globals():
        try:
          multiband = image
          if debug: print ("First band")
          if debug: print (multiband.bandNames().getInfo())
          len( multiband.bandNames().getInfo() )>0 
          emptyBand = False
        except:
          emptyBand = True
          print ("problem on 1st")

        # if len(multiband.bandNames().getInfo())==(f-startNumber+1):##checking if band was created by seeing if counter for batch and band number in batch add up 
        #   if debug: print ("band created")
      ##-----------------------------------------------------------------------------------------------------------
      #second image in batch and all others are added as bands to multiband image here
      else:
        try:
          multiband = ee.Image(multiband.addBands(image))
        except:
          if debug: print (multiband.bandNames().getInfo())
          if debug: print ("error adding new bands")

        # if len(multiband.bandNames().getInfo())==(f-startNumber+1):
        #   if debug: print ("band created")
      f += 1  #image counter - END of image loop in batch

    if staticExportRegion == True:
      exportRegion = defaultExportRegion #defined at input paramaters
    else:
      #exportRegion = ee.Geometry.Rectangle([-180, -90, 180, 90], None, False); # for global extent
      exportRegion = ee.Geometry.Rectangle([xminForMultiband, yminForMultiband, xmaxForMultiband, ymaxForMultiband], None, False); # for global extent
      if debug: print ("exportRegion coordinates: ", str(exportRegion.bounds().coordinates().getInfo()))
      #if debug: print ("fc check:",str(xminForMultiband),str(yminForMultiband),str(xmaxForMultiband),str(ymaxForMultiband))


    # creating output image name and adding suffixes depending on choice of parameters useful for filtering
    outName = str(tax)+prefix+str(output_scale_int)+"_run_"+str(i+startNumber)+"_hab"+habMapFormat+"_"+habMapLevel+"_elev"+elevFormat+"_"+suffix+"_"+scenario
    # if singleBandExports ==True:
    #   outName = multiband.select(0).bandNames()

    if debug: print (outName)
    assetId = targetImageCollName+"/"+outName
    if debug: print (assetId)
    if debug: print ("number of bands in image: ", str(len(multiband.bandNames().getInfo())))
    if toDrive:
      task = ee.batch.Export.image.toDrive(image= multiband,\
                                          description= outName,\
                                          folder=folderName,\
                                          scale= output_scale,\
                                          maxPixels=1e13,\
                                          region=exportRegion)
    else:
      task = ee.batch.Export.image.toAsset(image= multiband,\
                                          description= outName,\
                                          assetId=assetId,\
                                          scale= output_scale,\
                                          maxPixels=1e13,\
                                          region=exportRegion)
    if debug: print (assetId)
    
    #likely a better way to check if an asset exists (.data() functions etc) but this seems to work
    if ((toDrive==False) and (skipIfAssetExists==True) and (assetId in imageNames(ee.ImageCollection(targetImageCollName)))):
      if debug: print ("WARNING - ASSET EXISTS, SKIPPING EXPORTING")
    else:
      task.start()##code out if testing and dont want to export assets#
      if debug: print ("exporting multiband image"+ outName+"to drive ="+ str(toDrive))
    # if multiband in globals():
    del (multiband)
    if 'xminForMultiband' in globals(): del xminForMultiband 
    if 'xmaxForMultiband' in globals(): del xmaxForMultiband 
    if 'yminForMultiband' in globals(): del yminForMultiband 
    if 'ymaxForMultiband' in globals(): del ymaxForMultiband 
    if 'xminForFC' in globals(): del xminForFC
    if 'xmaxForFC' in globals(): del xmaxForFC
    if 'yminForFC' in globals(): del yminForFC
    if 'ymaxForFC' in globals(): del ymaxForFC

    pbar.write("Done with batch %i" % i)
    i+=1 #batch counter
    if debug: print ("deleting multiband variable to clear for next time")
  # Close the progress bar
  pbar.close()
  print("All Tasks done!")


['NA']
None


i 0
batch range(0, 2)
f 0
Processing feature no:  1
here
first feature habitats 200;201;300;305;400;405
chose to refine by habitats
Using wide format LUT
inVal 200;201;300;305;400;405
conc str list ['200', '201', '300', '305', '400', '405']
['200', '201', '300', '305', '400', '405']
level1 recoding habitat levels
habitats list [200, 300, 400]
max elev 4500
min elev -99999
refineByHabitatOption =  habRefine
habsPresentInHabMap =  True
refineByElevationOption =  elevRefine
refining by elevation
elevFormat =  image image
aggregating to coarser resolution
output scale chosen:  927.6624232772797
output scale integer:  927
exportRegion coordinates:  [[[-17.533778366754788, -9.956695562353477], [50.906248050297854, -9.956695562353477], [50.906248050297854, 21.01688033023245], [-17.533778366754788, 21.01688033023245], [-17.533778366754788, -9.956695562353477]]]
multiband not in globals
First band
['AVES_CR_22695207_1']
f 1
Processing feature no:  2
here
first feature habitats 600
chose to refi

##Show AOH on map - for debugging / QA/QC

DO needs improving and ideally adding to a code cell before/at start of Main processing pt 3 where could check outputs visually for species of interest before exporting.

In [ ]:
# new_image = ee.Image(spfilterMask.first())
# idsel = ['REPTILIA_LRlc_46585_1', 'REPTILIA_LRlc_5668_1', 'REPTILIA_LRcd_13053_1', 'REPTILIA_LRlc_46586_1'][3]# mising species checks
# idsel = "REPTILIA_NA_1000007459_1" # missing species checks
# new_image = ee.Image(sp_imageColl.filter(ee.Filter.eq("id",idsel)).first())
# new_image = ee.Image(spfilter.first())

# print (str(new_image.get("id").getInfo()))
# adding fields
# table = table.map(lambda feature:
#    feature.set(listField,splitID(feature.get(inputIdField),"_",2).cat("_").cat(splitID(feature.get(inputIdField),"_",3)))
#    )
# Import the Folium library
import folium

# Define a method for displaying Earth Engine image tiles to folium map.
def add_ee_layer(self, ee_image_object, vis_params, name):
  map_id_dict = ee.Image(ee_image_object).getMapId(vis_params)
  folium.raster_layers.TileLayer(
    tiles = map_id_dict['tile_fetcher'].url_format,
    attr = 'Map Data &copy; <a href="https://earthengine.google.com/">Google Earth Engine</a>',
    name = name,
    overlay = True,
    control = True
  ).add_to(self)

# Add EE drawing method to folium.
folium.Map.add_ee_layer = add_ee_layer

# Set visualization parameters.
vis_params = {
  'min': 0,
  'max': 1000,
  'palette': ["blue","red"]} #['006633', 'E5FFCC', '662A00', 'D8D8D8', 'F5F5F5']

# Create a folium map object.
my_map = folium.Map(location=[20, 0], zoom_start=3, height=500)

# Add the elevation model to the map object.
my_map.add_ee_layer(image, vis_params, 'new_image')

# Add a layer control panel to the map.
my_map.add_child(folium.LayerControl())

# Display the map.
display(my_map)


##AOH stats - fast 
NOTE: fast to run but awkward format - when going to drive it is GeoJSON - so needs work to reformat, currently done in R but could be done in pandas



In [ ]:
tableToDrive = True
suffix = "2015AD"
collection = "aoh_ic_pastureFillV2"
ic = ee.ImageCollection("projects/unep-wcmc/p08880_CCI_NbS/work_in_progress/"+collection).filter(ee.Filter.stringContains("system:index",suffix)).limit(10)
statsScale = ic.first().projection().nominalScale().getInfo()
exportRegion = ee.Geometry.Rectangle([-180, -90, 180, 90], None, False); # for global extent

# Area statistics function  - testing to see if any speed improvements from making bounds form a vector representation
def calc_aoh_stats_bounds (img):
  area_img = img.multiply(ee.Image.pixelArea()).divide(1e6).float()#.toUint32()
  tax1 = "blah"
  imgComb = img.reduce(ee.Reducer.sum())
  statsBounds = exportRegion#(imgComb.reduceToVectors(geometry=exportRegion ,geometryType = "bb", bestEffort = True)).geometry().bounds()
  sum = area_img.reduceRegion( 
    reducer = ee.Reducer.sum(), \
    geometry = statsBounds, \
    scale = statsScale, \
    maxPixels = 10e10)#.get("constant")
  #Return Fake point
  return ee.Feature(ee.Geometry.Point([0, 0]), {"id" : img.get("id")}).setMulti({"aoh_area": sum})#/*"filter" : tax1,*/

#print (ic.size())
#print (str(calc_aoh_stats_bounds(ic.first())))
outTable = ee.FeatureCollection(ic.map(calc_aoh_stats_bounds))#.aside(print)
print (str(ee.Dictionary(outTable.first().get("aoh_area")).getInfo()))

# Export.table.toDrive({
#     collection: outTable, 
#     description:suffix,
#     folder: collection,
#     #selectors:([field1,field2]),
#     fileFormat: "GeoJSON"
#   })


# Finally export the table to asset or drive respectively
if tableToDrive:
  task = ee.batch.Export.table.toDrive(collection = outTable, \
                                      description = suffix, \
                                      fileNamePrefix = suffix, \
                                      folder = collection, \
                                      fileFormat = 'GeoJSON'
                                      )
else:
  task = ee.batch.Export.table.toAsset(collection = outTable, #FIX THIS NOT TESTED
                                      description= outName,\
                                      assetId = outputAssetpath+"/"+outName
                                       )
task.start() 
print('Band names exported with stats.')


#Functions for AOH statistics -slow version






In [ ]:

# Area statistics function
def calc_aoh_stats(img):
  area_img = img.multiply(ee.Image.pixelArea()).divide(1e6).toFloat()
  sum = area_img.reduceRegion(
    reducer = ee.Reducer.sum(), \
    geometry = exportRegion, \
    scale = statsScale, \
    maxPixels = 10e10  
  ).get("constant")
  # Return Fake point
  return ee.Feature( ee.Geometry.Point([0, 0]), {"id" : img.get("id")}).setMulti({"filter" : tax1,"aoh_area": sum})

# Query a target bandname and get the image. Then compute for all bandnames
def to_image(element):
  id = ee.String(element)
  img = sp_all.select(id)
  return ee.Image(img).rename("constant").set("id",id.slice(0,30))#<check the slice bit of code doesnt truncate>

def iterfilter(current, prev): return ee.Algorithms.If(prev, ee.Image(prev).addBands(current),current)

def list_band_names(img):
    # Return Fake point\n","  
  return ee.Feature( ee.Geometry.Point([0, 0]), {"id" : img.get("id")}).set({"filter" : tax1})

# AOH Stats - slow version - Pt1 - filtering and preparing AOH data
Filters the imageCollection of the current output path to only those images with the target taxonomic group in format for getting stats (see below).
NB Slow to run but consistent output format, (one fc with a feature per AOH stat). So easy to use in other scripts.
Speed probably to do with the bit where it creates one large image collection with one AOH per image 

In [ ]:
####################################################################################################################
#Get the name of the run and the outputfolder
# First load all the images collections present in the given collection
# targetImageCollName = "projects/unep-wcmc/p8425_NatureMap/outputs/AOH_August2020/AOH_v1_multiband_habmap_v003"

# suffix = "" # suffix to be inserted as needed
# exportRegion = ee.Geometry.Rectangle([-180, -90, 180, 90], None, False); # for global extent

subString = ee.String(targetImageCollName).split("/").get(ee.String(targetImageCollName).split("/").size().subtract(1)).getInfo()
outputFolder = suffix ##'AOH_10km_mask/AOH_10km_mask_lvl2' # needs to be changed if saved to asset
# outputAssetpath = "projects/unep-wcmc/p8425_NatureMap/outputs/AOH_August2020/stats_habmap_anthro_elev_refine"
# outputAssetpath = "projects/unep-wcmc/p8425_NatureMap/outputs/AOH_August2020/stats_habmap_anthro_elev_refine"
# outputImageColl = "projects/unep-wcmc/p8425_NatureMap/outputs/AOH_August2020/stats_habmap_lvl1_elev_refine"
# print (str(subString.getInfo()))
listBandNamesOnly = False
tableToDrive = True # Overwriting here since I peferably want this saved to drive
exportingImage = False
imageToDrive = False # NB only exports if exportingImage==True
targetImageCollName = "users/alisoneyresrspb/CCI_NBS/work_in_progress/aoh_habmapV004_PNV_hyde" #alison's asset
factor = ee.Number(10000000000000/65535)
imageCollection = ee.ImageCollection(targetImageCollName).map(lambda image: image.divide(factor).uInt16())

print('A total of %i images in the imageCollection \'%s\' ' % (imageCollection.size().getInfo(), subString) )

#if want to do a test with a few bands
startBand = 0#
endBand = 100000 #set this to 100000 or so if no filtering to test

# ------------ #
# Parameters for checking
factor = 1 # at normal res use 1. For quick checks use higher value (i.e. coarser res) - it'll be inaccurate but can be useful
tax1 =  ['AMPHIBIA','MAMMALIA','AVES','REPTILIA','PLANTAE','reptiliaGARD','plantaeBGCI','CORALS',target_group][8] #tax#plantaeBGCI"##target_group # The target run group
filterByTax = True # Filter by taxon

# Filter by taxa if set
if filterByTax == True: 
  spfilter = ee.ImageCollection( imageCollection.filter(ee.Filter.stringContains("system:index",tax1)))
  # spfilter = ee.ImageCollection( spfilter.filter(ee.Filter.stringContains("system:index","lvl1").Not()))
  spfilter = ee.ImageCollection( spfilter.filter(ee.Filter.stringContains("system:index",suffix)))
  # spfilter = ee.ImageCollection(spfilter.filter(ee.Filter.neq("system:index","PLANTAE_stack_scale_11039_run_15")))
else: 
  spfilter = imageCollection
print('A total of %i images in the filtered imageCollection \'%s\' ' % (spfilter.size().getInfo(), subString) )

# ----------------------- #
imageScale = spfilter.first().projection().nominalScale()
imageScale = spfilter.first().projection().nominalScale()
print (imageScale.getInfo()) 
statsScale = ee.Number(imageScale).multiply(ee.Number(factor)).getInfo()
imageScaleInt = ee.Number(imageScale).toInt().getInfo()
statsScaleInt = ee.Number(statsScale).toInt().getInfo()

# Define the output name
outName = "stats_" +  "_" + tax1 + "_imageScale_" + str(imageScaleInt) + "m_statsScale_" + str(statsScaleInt) + "m_global" +suffix
# outName = "stats_" + subString + "_" + tax1 + "_imageScale_" + str(imageScaleInt) + "m_statsScale_" + str(statsScaleInt) + "m_global" +suffix
# Iter filter function. Could probably be converted into a lambda

# Create an image stack of the filtered images from the given collection
imgstacksp = spfilter.iterate( iterfilter, [])


sp_all = ee.Image(imgstacksp) 

#limit number for testing
sp_all =sp_all.slice(startBand,endBand)

sp_imgstacksplist = sp_all.getInfo()

# List of bandnames
sp_band_list = sp_all.bandNames()

sp_imageColl = ee.ImageCollection(sp_band_list.map(to_image))

# Now run the area calculation on each of the images
if listBandNamesOnly:
  fc_sp_stats = ee.FeatureCollection(sp_imageColl.map(list_band_names))
else:
  fc_sp_stats = ee.FeatureCollection(sp_imageColl.map(calc_aoh_stats))

#temp code--------------------------- use only if applyMaskMultiplicstion was False when creating the assets - as you dont want to do this twice
sp_imageColl.limit(10)
print ("blah",str(sp_imageColl.first().propertyNames().getInfo()))
print ("blah",str(ee.Image(sp_imageColl.first()).get("id").getInfo()))
print ("blah",str(sp_imageColl.first().select(0).bandNames().getInfo()))
spfilterMask = ee.ImageCollection(sp_imageColl.map(lambda image: image.multiply(image.mask()).rename(ee.String(image.get("id"))))) #set({"system:index":image.get("id")})
# print ("size of new image coll",str(spfilterMask.size().getInfo()))
imgstackspMask = spfilterMask.iterate( iterfilter, [])
sp_all = ee.Image(imgstackspMask)
#------------------------------------
# sp_all = ee.Image(imgstacksp)#if temp code above is run then annotate this line out

print ("first bandname",str(spfilterMask.first().select(0).bandNames().getInfo()))
# print ("After filtering imagecollection contains:", str(spfilterMask.aggregate_array("id").getInfo()))
print ("After filtering imagecollection contains:", str(spfilterMask.aggregate_array("system:index").getInfo()))
##---temp code end
# if debug:
  #print (str(sp_imageColl.aggregate_array("id").getInfo()))
print ("After filtering a total of {} images in the imageCollection".format(str(spfilter.aggregate_array("system:index").size().getInfo())))
print ("After filtering imagecollection contains:", str(spfilter.aggregate_array("system:index").getInfo()))
print('A total of %i images in the imageCollection %s' % (fc_sp_stats.size().getInfo(), "for AOH statistics") )
if listBandNamesOnly:
  print ("band names", str(fc_sp_stats.aggregate_array("id").getInfo()))

A total of 1375 images in the imageCollection 'aoh_habmapV004_PNV_hyde' 
A total of 532 images in the filtered imageCollection 'aoh_habmapV004_PNV_hyde' 
9276.620522123105
blah ['id', 'system:index', 'system:footprint', 'system:version', 'system:id', 'system:asset_size', 'system:bands', 'system:band_names']
blah AVES_CR_22695207_1
blah ['constant']
first bandname ['AVES_CR_22695207_1']
After filtering imagecollection contains: ['0', '1', '2', '3', '4', '5', '6', '7', '8', '9', '10', '11', '12', '13', '14', '15', '16', '17', '18', '19', '20', '21', '22', '23', '24', '25', '26', '27', '28', '29', '30', '31', '32', '33', '34', '35', '36', '37', '38', '39', '40', '41', '42', '43', '44', '45', '46', '47', '48', '49', '50', '51', '52', '53', '54', '55', '56', '57', '58', '59', '60', '61', '62', '63', '64', '65', '66', '67', '68', '69', '70', '71', '72', '73', '74', '75', '76', '77', '78', '79', '80', '81', '82', '83', '84', '85', '86', '87', '88', '89', '90', '91', '92', '93', '94', '95', '9

# AOH Stats - slow version - Pt2 - exporting stats for AOH as a table and/or the AOH images themselves
Takes output from Pt1, calculates area statistics and exports them. Option to export the multiband image (in exported tifs). The stats table is useful for  finding empty bands and cancbe used in extra filtering above (if exported as an asset). If exported to drive it is useful for linking bandids to bandnames (as tif files lose these when exported).

Slow to run but consistent output format, (one fc with a feature per AOH stat). So easy to use in other scripts.

In [ ]:
print("outName: {}".format(outName)  )
# Finally export the table to asset or drive respectively
if tableToDrive:
  task = ee.batch.Export.table.toDrive(collection = fc_sp_stats, \
                                      description = outName, \
                                      fileNamePrefix = outName, \
                                      folder = outputFolder, \
                                      fileFormat = 'CSV'
                                      )
else:
  task = ee.batch.Export.table.toAsset(collection = fc_sp_stats, #FIX THIS NOT TESTED
                                      description= outName,\
                                      assetId = outputAssetpath+"/"+outName
                                       )
task.start() 
print('Band names exported with stats.')

# export multiband image to asset or drive respectively (optional)
if exportingImage:
  outName2 = "image_" + subString + "_" + tax1 + "_imageScale_" + str(imageScaleInt) + "m_global" +suffix
  #print('Exporting a total of %i bands in multiband image for %s ' % (sp_all.bandNames().size().getInfo(),tax1) )
  if imageToDrive:
    task2 = ee.batch.Export.image.toDrive(image= sp_all,\
                                        description= outName2,\
                                        folder=outputFolder,\
                                        scale= imageScale.getInfo(),\
                                        maxPixels=1e13,\
                                        region=exportRegion)
    # if bandListToTable:
    #   task2 = ee.batch.Export.image.toDrive(table= table,\
    #                                       description= outName2,\
    #                                       folder=outputFolder)
    task2.start()
    print('multiband image exported')
  else:
    # task2 = ee.batch.Export.image.toAsset(image= image,\
    #                                     description= outputImageColl,\
    #                                     assetId=assetId,\
    #                                     scale= imageScale.getInfo(),\
    #                                     maxPixels=1e13,\
    #                                     region=exportRegion)
    # task2.start()
    # print('multiband image exported')
    print('code not set up for exporting image to drive currently')
  if debug:
    print (str(sp_imageColl.aggregate_array("id").getInfo()))

outName: stats__MultTax_imageScale_9276m_statsScale_9276m_globalmixlevelHabMapV004_1800
Band names exported with stats.


#Visualise on map (optional)




In [ ]:

new_image = ee.Image(template_image)
print (str(new_image.get("id").getInfo()))

# Import the Folium library
import folium

# Define a method for displaying Earth Engine image tiles to folium map.
def add_ee_layer(self, ee_image_object, vis_params, name):
  map_id_dict = ee.Image(ee_image_object).getMapId(vis_params)
  folium.raster_layers.TileLayer(
    tiles = map_id_dict['tile_fetcher'].url_format,
    attr = 'Map Data &copy; <a href="https://earthengine.google.com/">Google Earth Engine</a>',
    name = name,
    overlay = True,
    control = True
  ).add_to(self)

# Add EE drawing method to folium.
folium.Map.add_ee_layer = add_ee_layer

# Set visualization parameters.
vis_params = {
  'min': 0,
  'max': 1000,
  'palette': ["blue","red"]} #['006633', 'E5FFCC', '662A00', 'D8D8D8', 'F5F5F5']

# Create a folium map object.
my_map = folium.Map(location=[20, 0], zoom_start=3, height=500)

# Add the elevation model to the map object.
my_map.add_ee_layer(new_image.selfMask(), vis_params, 'new_image')

# Add a layer control panel to the map.
my_map.add_child(folium.LayerControl())

# Display the map.
display(my_map)

# fc1 = ee.FeatureCollection(fc.filter(ee.Filter.And((ee.Filter.eq(speciesId,id),ee.Filter.eq(seasonCode,id2)))))


None


# Export assets to drive 
# Export all assets to drive. To be run manually as needed. Exports all multiband images as a single multiband image NB code for bandnames needs checking


In [416]:
#-----------------------
#change only if want something different to the parameters at the start
# Get the name of the run and the outputfolder
inImageCollName = targetImageCollName #input image collection
outImageCollName = targetImageCollName #output iamge coll name if different #"projects/unep-wcmc/p8425_NatureMap/outputs/AOH_August2020/AOH_v1_multiband_habmap_v003"
# Get the name of the run and the outputfolder
subString = ee.String(inImageCollName).split("/").get(ee.String(inImageCollName).split("/").size().subtract(1)).getInfo()

outputFolder = folderName # set to folderName if want as defined in parameters at start of script, but can be changed
toDrive = False # set to True as preferably want this saved to drive. Could be cahgned to False if want exported to a different image collection though.
testing = False
useScaleFromImage = True # uses scale from each image else uses default scale
defaultScale = ee.ImageCollection(inImageCollName).first().projection().nominalScale().getInfo() #10000# ignored if 
# useRegionFromImage = True # uses scale from each image # <NOT YET IMPLEMENTED> uses output region from bounds of input image 
# ------------ #
# First load all the images collections present in the given collection
imageCollection = ee.ImageCollection(inImageCollName)
print('A total of %i images in the imageCollection \'%s\' ' % (imageCollection.size().getInfo(), subString) )

# Parameters for checking
# factor = 1 # at normal res use 1. For quick checks use higher value (i.e. coarser res) - it'll be inaccurate but can be useful
tax = "AVES" #target_group # The target run group - <to fix issues with naming for GARD and BGCI
print (tax)
filterByTax = True # Filter by taxon
startIndex = 0#startNumber #for filtering list i.e. running a subset
endIndex = 100000#exportMaxNumber+startNumber #for filtering list i.e. running a subset

# Filter by taxa if set
if filterByTax == True: 
  spfilter = ee.ImageCollection(imageCollection.filter(ee.Filter.stringContains("system:index",tax) ))
  # spfilter = ee.ImageCollection(imageCollection.filter(ee.Filter.stringContains("system:index","GARD")))
else: 
  spfilter = imageCollection
print (str(imageCollection.size().getInfo()))
# ----------------------- #
speciesKeepList spfilter.aggregate_array("system:index").getInfo()

print ((speciesKeepList))
speciesKeepList = speciesKeepList[startIndex:endIndex]
print (str(len(speciesKeepList)))


SyntaxError: ignored

In [ ]:

for num, imageName in enumerate(speciesKeepList, start=1):
  if useScaleFromImage: 
    output_scale = image.projection().nominalScale().getInfo()
  else:
    output_scale = defaultScale
  print ("output scale; {0}".format(str(output_scale)))
  print("test")
  image = ee.Image(targetImageCollName+"/"+imageName)
  outName = imageName
  if debug:
    print ('accessing image {0} {1}'.format(str(outName),str(num)))
  
# Finally export to asset or drive respectively

  if toDrive:
    task = ee.batch.Export.image.toDrive(image= image,\
                                        description= outName,\
                                        folder=outputFolder,\
                                        scale= output_scale,\
                                        maxPixels=1e13,\
                                        region=exportRegion)
    if bandListToTable:
      task2 = ee.batch.Export.image.toDrive(table= table,\
                                          description= outName,\
                                          folder=outputFolder)

  else:
    task = ee.batch.Export.image.toAsset(image= image,\
                                        description= outName,\
                                        assetId=assetId,\
                                        scale= output_scale,\
                                        maxPixels=1e13,\
                                        region=exportRegion)
  if testing:
    if ((skipIfAssetExists==True) and (outName in existsList)):
      if debug: print ("testing - not exporting NB asset exists")
    else:
      if debug: print ("testing - not exporting")
  else:
    if ((toDrive==False) and (skipIfAssetExists==True) and (outName in existsList)):
      if debug: print ("not exporting NB asset exists")
    else:
      task.start()###code out if testing and dont want to export assets
      if debug: print ("exporting multiband image"+ outName+"to drive ="+ str(toDrive))

# pbar.write("Done with batch %i" % i)
# i+=1 #batch counter
# if debug: print ("deleting multiband variable to clear for next time")
# Close the progress bar
# pbar.close()
print ("All Tasks done!")

IndentationError: ignored

##Export habitat map image to fractional image collection NEEDS FIXING - CRS EXPORT ERROR

In [ ]:
#/----------------------------------------------------------------------------------------------------------------
#Aim: creating a 1km image collection with aggregated 100m res habitat classes in each image - for use in AOH creation
#This should avoid as much unnecessary aggregation
#Elevation also aggregated to 1km - seems to not make a massive difference.

#Original created by: Andy Arnell 26/03/2020 - edited to align with a 30 degree global dataset
targetImageCollName = "projects/unep-wcmc/imported/habitats/jung_iucn_habMaps/Fractions_ht004_lvl2_30arcsec"#image collection
exportRegion = ee.Geometry.Rectangle([-180, -90, 180, 90], {}, False);#parameter for exporting
fractionMultiplier = 10000
habMap = ee.Image("users/Uploads/habitattypes/iucn_habitatclassification_composite_lvl2_ver004");#
#Map.addLayer(habMap.randomVisualizer(),"","habMap0",0,1)
#---------------------
template_image = ee.Image("Oxford/MAP/accessibility_to_cities_2015_v1_0") #potentially this will be removed soon as new accessibility to healthcare 2019 replaces it

output_scale = template_image.projection().nominalScale().getInfo()
print(output_scale)
print(template_image.projection().crs().getInfo())
reducerChoice = ee.Reducer.mean()

conc_num_list =ee.List([100,101,102,103,104,105,106,107,108,109,200,201,202,300,301,302,303,304,305,306,307,308,400,401,402,403,404,405,406,407,500,501,502,503,504,505,506,507,508,509,510,511,513,514,515,516,517,600,800,801,802,803,900,901,908,909,
1000,1001,1002,1003,1004,1100,1101,1102,1103,1104,1105,1106,1200,1206,1207,1400,1401,1402,1403,1404,1405])

# functions
# Takes an input image and reprojects it in resolution and projection to a provided template image
def reprojectImage (image,template_image,reducer):
  targProj = template_image.projection()
  inProj = image.projection()
  image = image.reproject(crs = inProj)
  res = image.reproject(image.projection()).reduceResolution(reducer = reducer, maxPixels = 65024).reproject(crs = targProj)
  return res

###NB check
def imageCollFromValsReproject(number):
  return reprojectImage(ee.Image(habMap.eq(ee.Number(number))),template_image,reducerChoice) \
  .toUint16().multiply(fractionMultiplier).set("system:index",str(imagePrefixInHabMapImageColl+str(number)))
  #.rename(ee.String(number))}

#processing
imageColl = ee.ImageCollection(conc_num_list.map(imageCollFromValsReproject))
print (str(imageColl.aggregate_array(imagePrefixInHabMapImageColl).getInfo()))

#Map.addLayer(imageColl.sum())
#Map.addLayer(habMap.randomVisualizer())
#exporting images to image collection
assetId = str(targetImageCollName+"/"+htclass)
print (assetId)
for f in conc_num_list.getInfo()[0:1]:
  #print (f)
  htclass = str(imagePrefixInHabMapImageColl+str(f))#conc_num_list.getInfo()[f]
  print (htclass)
  exportImage = ee.Image(imageColl.filter(ee.Filter.eq("system:index",htclass)).first())
  task = ee.batch.Export.image.toAsset(image = exportImage, \
      description = htclass,\
      assetId = assetId, \
      scale = output_scale,\
      maxPixels = 1e13, \
      region = exportRegion)
  task.start()
  print ("exporting")
  f+=1


#checking
# # Set visualization parameters.
# vis_params = {
#   'min': 0,
#   'max': 1000,
#   'palette': ["blue","red"]} #['006633', 'E5FFCC', '662A00', 'D8D8D8', 'F5F5F5']

# new_image = (reprojectImage(habMap.eq(101),template_image,reducerChoice))

new_image = ee.Image(imageColl.filter(ee.Filter.eq("system:index","htclass101")).first())
# Create a folium map object.
my_map = folium.Map(location=[20, 0], zoom_start=3, height=500)

# Add the elevation model to the map object.
my_map.add_ee_layer(new_image, vis_params, 'new_image')

# Add a layer control panel to the map.
my_map.add_child(folium.LayerControl())

# Display the map.
display(my_map)

927.6620522123104
EPSG:4326
[]
projects/unep-wcmc/imported/habitats/jung_iucn_habMaps/Fractions_ht004_lvl2_30arcsec/htclass100
htclass100
exporting


EEException: ignored

##Change maps


In [ ]:
import ee
# import geemap

# Map = geemap.Map()

#probably needs converting to python to be more reliable - crashes currently a fair bit
######################################################################################################################
#importing imagecollection
baselineYear = "1996"#keep the same
timeStepList = ["2016"]#,"2007","2008","2009","2010","2015","2016"]#note that tax filter should be ".Not()" for global

table = ee.FeatureCollection("projects/unep-wcmc/p08685_UNEP_mangroves/raw/comp_list_species_list_inc_inverts");#didnt use inverts in the end but fine to use as filtering already done at AOH stage using LUT
sp_imageColl_multiband = ee.ImageCollection("projects/unep-wcmc/p08685_UNEP_mangroves/work_in_progress/AOH_v2_multiband_habmap_v004")

exportRegion = ee.Geometry.Rectangle([-180, -55, 180, 55], {}, False);#geometry2
factor = 1#multiplication
toDrive = False #set to True or False if want asset or drive
driveFolder = "GMW_MultiTax"
assetFolder = "projects/unep-wcmc/p08685_UNEP_mangroves/work_in_progress/rerun2021/AOH_v2_1_habmap_v004_rwr_change"
outPrefix = "GMW_MultiTax"
areaCorrectionFactor = 1e8

# # #functions
# def joinToLUT(ic,lut,xfield,yfield):
#   pF = ee.FeatureCollection (ic);# Create the primary collection.
#   sF = ee.FeatureCollection(lut);# Create the secondary collection.
#   ftFilter = ee.Filter.equals({'leftField':xfield, 'rightField':yfield})
#   innerJoin = ee.Join.inner('primary', 'secondary');# Define the join.
#   ftJoin = innerJoin.apply(pF, sF, ftFilter);# Apply the join.
#   print('Inner join to ft: ' + 'ft table to fc:', ftJoin.size()); # Print the result.
# # Make a polygon feature collection from the joined data results.

# def func_grv(pair):
#   pF = ee.Feature(pair.get('primary'))
#   sF = ee.Feature(pair.get('secondary'))
#   return pF.set(sF.toDictionary())

#   fcJoined = (ftJoin.map(func_grv))

# #))
# return fcJoined

def apply_rr(img):
  rr = ee.Image(ee.Number(img.get(areaField)))
  img1 = (img.multiply(ee.Image.pixelArea()).divide(areaCorrectionFactor)).divide(rr)
  img2 = img1.copyProperties(img)
  return ee.Image(img2)

def loss_t0_t1(image):
  id = image.get("id_no")
  t0_area_img = image.multiply(ee.Image.pixelArea()).divide(areaCorrectionFactor)
  t1_img0 = sp_imageColl_t1.filter(ee.Filter.eq("id_no",id)).first()
  t1_img = t1_img0.where(t1_img0.gt(image),image)#as only want loss
  t1_area_img = t1_img.multiply(ee.Image.pixelArea()).divide(areaCorrectionFactor)
  t0_global_area_val = ee.Number(sp_fcol_t0_global.filter(ee.Filter.eq("id_no",id)).first().get("aoh_area_t0_global"))
  return ((t1_area_img.subtract(t0_area_img)).divide(t0_global_area_val)).set("id_no",id)

def change_t0_t1(image):
  id = image.get("id_no")
  t0_area_img = image.multiply(ee.Image.pixelArea()).divide(areaCorrectionFactor)
  t1_img = sp_imageColl_t1.filter(ee.Filter.eq("id_no",id)).first()
  t1_area_img = t1_img.multiply(ee.Image.pixelArea()).divide(areaCorrectionFactor)
  t0_global_area_val = ee.Number(sp_fcol_t0_global.filter(ee.Filter.eq("id_no",id)).first().get("aoh_area_t0_global"))
  return ((t1_area_img.subtract(t0_area_img)).divide(t0_global_area_val)).set("id_no",id)

#function to map id_no to each imageColl
def imageIDSplitToIDNO(feature):
  return feature.set("id_no_int",ee.Number.parse(ee.String(feature.get("id_no")).split("_").get(2)).toInt64())

def filterNameByString(fc):
  return fc.filter(ee.Filter.stringContains("system:index","mangOnly_"+year))


#k
# for k in range(0, timeStepList.length, 1):
# year = timeStepList[k]

year = timeStepList[0]  #<-change the value in brackets to change year

sp_fcol_t0_global = ee.FeatureCollection("projects/unep-wcmc/p08685_UNEP_mangroves/work_in_progress/fc_MultiTax_stats_1103m_global_1996")
sp_fcol_t0 = ee.FeatureCollection("projects/unep-wcmc/p08685_UNEP_mangroves/work_in_progress/fc_MultiTax_stats_1103m_aoi_bounds_1996")
sp_fcol_t1 = ee.FeatureCollection("projects/unep-wcmc/p08685_UNEP_mangroves/work_in_progress/fc_MultiTax_stats_1103m_aoi_bounds_"+year)

#select relevant year
var sp_imageColl_multiband_t0 = fitlerNameByString("mangOnly_"+str(baselineYear))
var sp_imageColl_multiband_t1 = fitlerNameByString("mangOnly_"+str(year))
var sp_imageColl_multiband_t0 = sp_imageColl_multiband_t0.map(lambda image: image.toFloat()))
var sp_imageColl_multiband_t1 = sp_imageColl_multiband_t1.map(lambda image: image.toFloat()))

#--------------------------------------------------------------------------------------------
scale = sp_imageColl_multiband_t0.first().projection().nominalScale().multiply(factor).getInfo()
intScale = ee.Number(scale).toInt().getInfo()
# #-------------------------------------------------------------------------------

# t0
# prep for AOH area feature collection
# filter to remove unwanted features
# sp_fcol_t0_global = filterUnwanted(sp_fcol_t0_global)
print ("SIZE sp_fcol_t0_global",str(sp_fcol_t0_global.size()))
#changing field names

def changeFieldNames (img):
  return img.select(["id_no","aoh_area"],["id_no","aoh_area_t0_global"])}:
  
sp_fcol_t0_global = sp_fcol_t0_global.map(changeFieldNames)

#print ("t0_global",sp_fcol_t0_global.limit(10))

# #---------------------------------------------------
# Create an image stack of sp images
imgstacksp_t0 = ee.Image(sp_imageColl_multiband_t0.iterate(function(current, prev){
  ans = ee.Algorithms.If(prev, ee.Image(prev).addBands(current),current)
  return ans
  }, []))
#make each band into an image in an image collection
def to_image_t0 (element):
  id = ee.String(element)
  img = imgstacksp_t0.select(id)
  return ee.Image(img).rename("constant").set("id_no",id.slice(0,50))
# sp_imageColl = ee.ImageCollection(sp_index_list.map(to_image))
sp_imageColl_t0 = ee.ImageCollection((imgstacksp_t0.bandNames()).map(to_image_t0))

#filter the collection to remove unwanted images
#sp_imageColl_t0 = filterUnwanted(sp_imageColl_t0)
#print ("SIZE sp_imageColl_t0",sp_imageColl_t0.size())
#print ("count agg",sp_imageColl_t0.aggregate_array("id_no"))
#Map.addLayer(sp_imageColl_t0.sum(),  {min: 1, max: 500016, palette: ['blue','green','orange','red']},"sum_sp_t0",0,1)
# #----------------------------------------------------------
imageColl_t0 = ee.ImageCollection(joinToLUT(sp_imageColl_t0,sp_fcol_t0_global,"id_no","id_no")).map(imageIDSplitToIDNO)
print (imageColl_t0.limit(10))
imageColl_t0 = ee.ImageCollection(joinToLUT(imageColl_t0,table,"id_no_int","internalTaxonId"))
areaField = "aoh_area_t0_global"
imageColl_t0 = imageColl_t0.map(apply_rr)
# #------------------------------------------------------------------
#t1
#---------------------------------------------------------------------
#prep for AOH area feature collection
#getting area outside AOI in t0
#join global to AOI for t0, and calculate difference
#filter to remove unwanted features
# sp_fcol_t0 = filterUnwanted(sp_fcol_t0)
print ("SIZE sp_fcol_t0",sp_fcol_t0.size())
sp_fcol_t0_notAOI = ee.FeatureCollection(joinToLUT(sp_fcol_t0,sp_fcol_t0_global,"id_no","id_no"))

def func_qsa(img)return img.set("aoh_area_notAOI",(ee.Number(img.get("aoh_area_t0_global")).subtract(ee.Number(img.get("aoh_area"))))).select(["id_no","aoh_area_notAOI"])}:
sp_fcol_t0_notAOI = sp_fcol_t0_notAOI.map(function(img){return img.set("aoh_area_notAOI",(ee.Number(img.get("aoh_area_t0_global")).subtract(ee.Number(img.get("aoh_area"))))).select(["id_no","aoh_area_notAOI"])}
sp_fcol_t0_notAOI = sp_fcol_t0_notAOI.map(func_qsa)

print ("t0_notAOI",sp_fcol_t0_notAOI.limit(10))
#getting global area for t1
#join t1 to area outside AOI and sum together
# sp_fcol_t1 = filterUnwanted(sp_fcol_t1)
print ("SIZE sp_fcol_t1",sp_fcol_t1.size())

sp_fcol_t1_global = ee.FeatureCollection(joinToLUT(sp_fcol_t1,sp_fcol_t0_notAOI,"id_no","id_no"))
print ("t1_global",sp_fcol_t1_global.limit(100))

def func_doj(img)return img.set("aoh_area_t1_global",(ee.Number(img.get("aoh_area_notAOI")).add(ee.Number(img.get("aoh_area")))))}:
sp_fcol_t1_global = sp_fcol_t1_global.map(function(img){return img.set("aoh_area_t1_global",(ee.Number(img.get("aoh_area_notAOI")).add(ee.Number(img.get("aoh_area")))))}
sp_fcol_t1_global = sp_fcol_t1_global.map(func_doj)

print ("t1_global",sp_fcol_t1_global.limit(100))
# sp_fcol_t1_global = ee.FeatureCollection(joinToLUT(sp_fcol_t0_global,sp_fcol_t0,"id_no","id_no"))

#AOH area feature collection
#filter to remove unwanted features
# sp_fcol_t1_global = filterUnwanted(sp_fcol_t1_global)
print ("SIZE sp_fcol_t1_global",sp_fcol_t1_global.size())
#selecting fields

def func_bsc (img) return img.select(["id_no","aoh_area_t1_global"])}:
sp_fcol_t1_global = sp_fcol_t1_global.map(function (img) {return img.select(["id_no","aoh_area_t1_global"])}
sp_fcol_t1_global = sp_fcol_t1_global.map(func_bsc)

print ("t1_global",sp_fcol_t1_global.limit(10))
####/-------------------------------------------------
# Create an image stack of sp images
imgstacksp_t1 = ee.Image(sp_imageColl_multiband_t1.iterate(function(current, prev){
  ans = ee.Algorithms.If(prev, ee.Image(prev).addBands(current),current)
  return ans
  }, []))
#make each band into an image in an image collection
def to_image_t1 (element):
  id = ee.String(element)
  img = imgstacksp_t1.select(id)
  return ee.Image(img).rename("constant").set("id_no",id.slice(0,50))

sp_imageColl_t1 = ee.ImageCollection((imgstacksp_t1.bandNames()).map(to_image_t1))
# print (sp_imageColl_t1.size(),"SIZE sp_imageColl_t1")
#filter the collection to remove unwanted images
# sp_imageColl_t1 = filterUnwanted(sp_imageColl_t1)
print ("SIZE sp_imageColl_t1",sp_imageColl_t1.size())
#Map.addLayer(sp_imageColl_t1.sum(),  {min: 1, max: 500016, palette: ['blue','green','orange','red']},"sum_sp_t1",0,1)
# #----------------------------------------------------------
imageColl_t1 = ee.ImageCollection(joinToLUT(sp_imageColl_t1,sp_fcol_t1_global,"id_no","id_no"))
print (imageColl_t1.limit(10))

areaField = "aoh_area_t1_global"
imageColl_t1 = ee.ImageCollection(imageColl_t1.map(apply_rr)).map(imageIDSplitToIDNO)
imageColl_t1 = ee.ImageCollection(joinToLUT(imageColl_t1,table,"id_no_int","internalTaxonId"))
print (imageColl_t1,"imageColl_t1_final")
# rwr_t1 = imageColl_t1.sum()#.clip(aoi)
#------------------------------------
#--------------------------------
loss_sp_imageColl_t0_t1 = ee.ImageCollection(sp_imageColl_t0.map(loss_t0_t1)).map(imageIDSplitToIDNO)
loss_sp_imageColl_t0_t1 = ee.ImageCollection(joinToLUT(loss_sp_imageColl_t0_t1,table,"id_no_int","internalTaxonId"))
print ("loss_sp_imageColl_t0_t1 size",loss_sp_imageColl_t0_t1.size())
sp_loss_t0_t1 = loss_sp_imageColl_t0_t1.sum()#.selfMask()#.clip(aoi)
# sp_loss_t0_t1_log10 = sp_loss_t0_t1.selfMask().multiply(ee.Number(-100000)).log10()

change_sp_imageColl_t0_t1 = ee.ImageCollection(sp_imageColl_t0.map(change_t0_t1)).map(imageIDSplitToIDNO)
# print ("change_sp_imageColl_t0_t1",change_sp_imageColl_t0_t1.limit(10))
change_sp_imageColl_t0_t1 = ee.ImageCollection(joinToLUT(change_sp_imageColl_t0_t1,table,"id_no_int","internalTaxonId"))
#change_sp_imageColl_t0_t1 = change_sp_imageColl_t0_t1.filter(ee.Filter.Or(ee.Filter.eq("className","AMPHIBIA"),ee.Filter.eq("className","AVES"),(ee.Filter.eq("className","MAMMALIA"),(ee.Filter.eq("className","REPTILIA")))))
# print (change_sp_imageColl_t0_t1.aggregate_array("className").distinct())
print ("change_sp_imageColl_t0_t1 size",change_sp_imageColl_t0_t1.size())
# print ("change_sp_imageColl_t0_t1",change_sp_imageColl_t0_t1.limit(10))

splitList = imageColl_t0.aggregate_array("className").distinct().sort().getInfo()
print (splitList)
# splitList = ["GASTROPODA"]

# print(sp_fcol_t0.reduceColumns(ee.Reducer.sum(),["aoh_area"]))
# print(sp_fcol_t1.reduceColumns(ee.Reducer.sum(),["aoh_area"]))
# print(sp_fcol_t1_global.reduceColumns(ee.Reducer.sum(),["aoh_area_t1_global"]))

j
for j in range(0, splitList.length, 1):
  filterName = splitList[j]
  #filter by group
  imageColl_t0_sel = imageColl_t0.filter(ee.Filter.eq("className",filterName))
  imageColl_t1_sel = imageColl_t1.filter(ee.Filter.eq("className",filterName))
  change_sp_imageColl_t0_t1_sel = change_sp_imageColl_t0_t1.filter(ee.Filter.eq("className",filterName))
  #reduce each outputs to an image for that group
  rwr_t1 = ee.Image(imageColl_t1_sel.sum().setMulti({"taxa":filterName,"baselineYear":baselineYear,"year":year}))#.clip(aoi)
  rwr_t0 = ee.Image(imageColl_t0_sel.sum().setMulti({"taxa":filterName,"baselineYear":baselineYear,"year":baselineYear}))#.clip(aoi)
  sp_change_t0_t1 = ee.Image(change_sp_imageColl_t0_t1_sel.sum()).setMulti({"taxa":filterName,"year":year,"baselineYear":baselineYear})#.clip(aoi)
  #make a list to export
  outputsList = [rwr_t0,rwr_t1,sp_change_t0_t1]#, sp_loss_t0_t1, sp_loss_t0_t1_log10]
  outNameList = ["rwr_t"+baselineYear,"rwr_t"+year,"sp_change_t"+baselineYear+"_t"+year];#,"sp_loss_t0_t"+year,"sp_loss_t0_t"+year+"_log10"]

  outputsList = [outputsList[2]]#change index to run - as list hangs if run in full
  outNameList = [outNameList[2]]#change index to run - as list hangs if run in full

  # outputsList = [rwr_t0,rwr_t1,sp_change_t0_t1]#, sp_loss_t0_t1, sp_loss_t0_t1_log10]
  # outNameList = ["rwr_t"+baselineYear,"rwr_t"+year,"sp_change_t"+baselineYear+"_t"+year];#,"sp_loss_t0_t"+year,"sp_loss_t0_t"+year+"_log10"]
  print ("list check - if val 1 then all ok: ",outputsList.length/outputsList.length)
  # print ("outNameList length: ",outNameList.length())
  #print (filterName,year,"change imageColl size",change_sp_imageColl_t0_t1_sel.size())
  Map.addLayer(ee.Image(sp_change_t0_t1))
  i
  for i in range(0, outputsList.length, 1):
    outFullName = outPrefix+"_"+outNameList[i]+"_scale_"+intScale+"m"+"_"+filterName
    print (outFullName)
    print (outputsList[i])
    # print (outputsList[i].get("baselineYear"))
    # print (outputsList[i].get("taxa"))
    print (outNameList[i])
    if (toDrive){

      Export.image.toDrive(
          {
          'image': outputsList[i],
          'description': "drive_"+outFullName,
          'folder': driveFolder,
          'scale': scale,
          'maxPixels': 1e13,
          'region': exportRegion,#.bounds(),
          })
    }
    else {
      Export.image.toAsset(
          {
          'image': outputsList[i],
          'description':"asset_"+outFullName,
          'assetId': assetFolder+"/"+outFullName,
          'scale': scale,
          'maxPixels': 1e13,
          'region': exportRegion,#.bounds(),
          })
    }
# Map


SyntaxError: ignored